In [1]:
import io
import json
import os
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload


# ======================================================
# 1) Credenciales (Colab usa MI_JSON desde userdata; fuera de Colab usa MI_JSON env)
# ======================================================
def get_credentials() -> Credentials:
    """
    - En Colab: usa MI_JSON desde google.colab.userdata (NO os.environ)
    - En GitHub/local: usa variable de entorno MI_JSON
    """
    info: Dict

    try:
        from google.colab import userdata  # type: ignore

        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado en Colab userdata")
        info = json.loads(mi_json)
        print("Entorno detectado: Google Colab")
    except Exception:
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado como variable de entorno (GitHub/local)")
        info = json.loads(mi_json)
        print("Entorno detectado: GitHub / local")

    return Credentials.from_service_account_info(
        info,
        scopes=[
            "https://www.googleapis.com/auth/drive.readonly",
            "https://www.googleapis.com/auth/spreadsheets.readonly",
        ],
    )


creds = get_credentials()
drive_service = build("drive", "v3", credentials=creds)


# ======================================================
# 2) Helpers: meses en español + parsing del nombre del archivo
# ======================================================
MES_MAP = {
    "ene": 1,
    "feb": 2,
    "mar": 3,
    "abr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "ago": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dic": 12,
}
MES_NOMBRE = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}


def sheet_name_from_date(dt: datetime) -> str:
    """Ej: datetime(2025,12,...) -> 'Diciembre 2025' """
    return f"{MES_NOMBRE[dt.month]} {dt.year}"


def parse_range_from_filename(name: str) -> Optional[Tuple[int, int, int, int]]:
    """
    Espera nombres tipo:
      'Asignaciones de Cartera Ene26-Abr26.xlsx'
      'Asignaciones de Cartera Sep25-Dic25.xlsx'
    Retorna (start_year, start_month, end_year, end_month) o None si no matchea.
    """
    m = re.search(r"([A-Za-z]{3})(\d{2})\s*-\s*([A-Za-z]{3})(\d{2})", name, flags=re.IGNORECASE)
    if not m:
        return None

    m1, y1, m2, y2 = m.group(1).lower(), m.group(2), m.group(3).lower(), m.group(4)
    if m1 not in MES_MAP or m2 not in MES_MAP:
        return None

    start_month = MES_MAP[m1]
    end_month = MES_MAP[m2]
    start_year = 2000 + int(y1)
    end_year = 2000 + int(y2)

    return (start_year, start_month, end_year, end_month)


def month_index(year: int, month: int) -> int:
    """Convierte (year, month) a índice comparable."""
    return year * 12 + month


def file_covers_month(file_range: Tuple[int, int, int, int], target_dt: datetime) -> bool:
    sy, sm, ey, em = file_range
    t = month_index(target_dt.year, target_dt.month)
    a = month_index(sy, sm)
    b = month_index(ey, em)
    return a <= t <= b


def is_df_empty_like(df: Optional[pd.DataFrame]) -> bool:
    """
    Considera 'vacía' si:
    - df es None
    - df tiene 0 filas
    - o todas las filas están completamente NaN
    """
    if df is None or df.shape[0] == 0:
        return True
    return df.dropna(how="all").shape[0] == 0


# ======================================================
# 3) Listar archivos en carpeta y elegir el correcto por mes
# ======================================================
def list_assignment_files_in_folder(folder_id: str) -> List[Dict]:
    """
    Lista archivos en la carpeta cuyo nombre contenga 'Asignaciones de Cartera'
    y devuelve una lista con metadatos: id, name, mimeType, modifiedTime, parsed_range
    """
    q = f"'{folder_id}' in parents and trashed=false and name contains 'Asignaciones de Cartera'"
    files: List[Dict] = []
    page_token = None

    while True:
        resp = (
            drive_service.files()
            .list(q=q, fields="nextPageToken, files(id,name,mimeType,modifiedTime)", pageToken=page_token)
            .execute()
        )

        for f in resp.get("files", []):
            fr = parse_range_from_filename(f.get("name", ""))
            if fr:
                f["parsed_range"] = fr
                files.append(f)

        page_token = resp.get("nextPageToken")
        if not page_token:
            break

    if not files:
        raise ValueError(
            "No encontré archivos 'Asignaciones de Cartera' con rango tipo Ene26-Abr26 dentro de la carpeta."
        )

    return files


def pick_file_for_month(files_meta: List[Dict], target_dt: datetime) -> Dict:
    """
    Escoge el archivo cuya ventana (en el nombre) cubra el mes target_dt.
    Si hay varios, elige el de rango más corto (más específico) y si empatan, el más reciente.
    """
    candidates = []
    for f in files_meta:
        fr = f["parsed_range"]
        if file_covers_month(fr, target_dt):
            sy, sm, ey, em = fr
            span = month_index(ey, em) - month_index(sy, sm)
            candidates.append((span, f.get("modifiedTime", ""), f))

    if candidates:
        candidates.sort(key=lambda x: (x[0], x[1]))  # menor span, luego por modifiedTime asc
        min_span = candidates[0][0]
        same_span = [c for c in candidates if c[0] == min_span]
        same_span.sort(key=lambda x: x[1], reverse=True)  # más reciente primero
        return same_span[0][2]

    t = month_index(target_dt.year, target_dt.month)

    past = []
    for f in files_meta:
        sy, sm, ey, em = f["parsed_range"]
        end_i = month_index(ey, em)
        if end_i <= t:
            past.append((end_i, f.get("modifiedTime", ""), f))
    if past:
        past.sort(key=lambda x: (x[0], x[1]), reverse=True)
        return past[0][2]

    future = []
    for f in files_meta:
        sy, sm, ey, em = f["parsed_range"]
        start_i = month_index(sy, sm)
        if start_i >= t:
            future.append((start_i, f.get("modifiedTime", ""), f))
    if future:
        future.sort(key=lambda x: (x[0], x[1]))
        return future[0][2]

    raise ValueError("No se pudo escoger un archivo por fecha (revisa nombres/rangos).")


# ======================================================
# 4) Descargar archivo (Google Sheets o Excel) a memoria
# ======================================================
def download_file_to_buffer(file_id: str, mime_type: str) -> io.BytesIO:
    buffer = io.BytesIO()

    if mime_type == "application/vnd.google-apps.spreadsheet":
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        )
    else:
        request = drive_service.files().get_media(fileId=file_id)

    downloader = MediaIoBaseDownload(buffer, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()

    buffer.seek(0)
    return buffer


# ======================================================
# 5) Intentar leer hoja del mes target; si está vacía -> retroceder mes a mes
# ======================================================
def shift_month(dt: datetime, n: int) -> datetime:
    """Mueve dt n meses (n puede ser negativo)."""
    y = dt.year + (dt.month - 1 + n) // 12
    m = (dt.month - 1 + n) % 12 + 1
    return datetime(y, m, 1)


def load_assignment_base_from_folder(
    folder_id: str, max_back_months: int = 24
) -> Tuple[pd.DataFrame, Dict, str]:
    """
    Busca base del mes actual; si no existe o está vacía,
    busca mes anterior (incluyendo cambio de archivo si aplica).
    Retorna: (df, file_meta, sheet_name_usada)
    """
    files_meta = list_assignment_files_in_folder(folder_id)
    today = datetime.today()

    last_error: Optional[Exception] = None

    for back in range(0, max_back_months + 1):
        target_dt = shift_month(today, -back)
        target_sheet = sheet_name_from_date(target_dt)
        chosen = pick_file_for_month(files_meta, target_dt)

        try:
            buffer = download_file_to_buffer(chosen["id"], chosen["mimeType"])
            df = pd.read_excel(buffer, sheet_name=target_sheet, engine="openpyxl")

            if is_df_empty_like(df):
                print(f"🟡 {target_sheet} encontrado pero vacío en: {chosen['name']} -> probando mes anterior...")
                continue

            print("✅ Base encontrada")
            print(f"   Archivo: {chosen['name']}")
            print(f"   Hoja:    {target_sheet}")
            return df, chosen, target_sheet

        except Exception as e:
            last_error = e
            print(
                f"🟠 No se pudo usar {target_sheet} en {chosen['name']} ({type(e).__name__}) -> probando mes anterior..."
            )

    raise RuntimeError(
        f"No encontré una hoja válida en los últimos {max_back_months} meses. "
        f"Último error: {repr(last_error)}"
    )


# ======================================================
# 6) EJECUCIÓN
# ======================================================
FOLDER_ID = "1cf2p3R7iM0xowAt4muEruDwxZoZqD_jB"

df, meta_file, sheet_used = load_assignment_base_from_folder(
    folder_id=FOLDER_ID,
    max_back_months=24,
)

df.head()

Entorno detectado: GitHub / local


✅ Base encontrada
   Archivo: Asignaciones de Cartera Ene26-Abr26.xlsx
   Hoja:    Febrero 2026


,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,Deuda Resuelve,DBT,...,Bucket,Negociador,Tipo Elegible,PotCred_num,PotCred_cat,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level
0,3106653868,3763762,Jorge Luis Camargo Fontalvo,899359,jlc5905@gmail.com,6,CREDIVALORES,401093039,3996300.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,1176.0,Atrasado,2025-01-21,380.0,Prioridad 1
1,3106653868,3763762,Jorge Luis Camargo Fontalvo,899360,jlc5905@gmail.com,6,Serfinanza,543000000,5763450.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,NaN,Al día,2025-01-21,380.0,Prioridad 2
2,3173740082,1034308242,Nailen Josefina Bahamon Sandoval,1255027,nai.bs@hotmail.com,1,Falabella,300002000000,13506762.0,17310912.0,...,0,Lina Maria Peña Gordillo,NaN,NaN,NaN,0.0,Al día,2024-10-25,468.0,Prioridad 1
3,3224821336,1093755862,Diego Andres Delgado Galvis,1275933,diegodlgd90@gmail.com,1,Davivienda,402516857,10257450.0,10257450.0,...,4,Lina Maria Peña Gordillo,NaN,NaN,NaN,196.0,Atrasado,NaT,NaN,Prioridad 2
4,3167086550,37826977,Luz Mery Quintero Blanco,1297740,luzmb.1225@gmail.com,4,Serfinanza,542060188,8083900.0,91913600.0,...,2,Angie Natalia Borda Soler,NaN,NaN,NaN,483.0,Atrasado,2024-04-30,646.0,Prioridad 1


In [2]:
df = df.rename(columns={"Deuda Resuelve": "D_BRAVO"})

In [3]:
df = df.rename(columns={"Meses de atraso": "MORA"})

In [4]:
df.columns

Index(['Referencia', 'Cedula', 'Nombre del cliente', 'Id deuda', 'correo',
       'Deudas Activas', 'Banco', 'Número de Crédito', 'D_BRAVO', 'DBT',
       'MORA', 'Dias de Atraso', 'Apartado Mensual', 'Fecha inicio',
       'vehiculo', 'Meses en el Programa', 'tipo_cliente', 'Comisión Mensual',
       'Tipo de cobro', 'CE', 'Estado Deuda', 'estado_novacion', 'Fecha PL',
       'sub_estado_deuda', 'ID_reparadora', 'estado_reparadora',
       'sub_estado_reparadora', 'dias_mora_ingreso', 'ultima_actividad',
       'PB_PL', 'estado_estructuracion', 'estado_flujo_liquidacion',
       'Ahorro total', 'Ahorro medio', 'Por cobrar', 'ultimo contacto',
       'fecha mensaje', 'Descuento', 'Mora_estructurado', 'MORA_CREDITO',
       'Potencial Credito', 'ultimo Pab', 'ultima act', 'Tipo de Liquidacion',
       'BANCOS_ESTANDAR', 'Clasificación BANCO', 'Apartados 30%', 'Plazo',
       'Estructurable', 'Potencial', 'Bucket', 'Negociador', 'Tipo Elegible',
       'PotCred_num', 'PotCred_cat', 'Días

In [5]:
# ---------------------------------------
# PASO 1 — Construir base de cartera asignada
# ---------------------------------------

# Trabajamos sobre copia por seguridad
df_cartera = df.copy()

# Columnas que queremos conservar
cols_cartera = [
    "Referencia",
    "Id deuda",
    "Cedula",
    "Nombre del cliente",
    "Negociador",
    "BANCOS_ESTANDAR",
    "Descuento",
    "D_BRAVO",
    "MORA",
    "Estructurable",
    "Potencial",
    "Meses en el Programa",
    "Tipo de Liquidacion",
    "Bucket",
    "Ahorro total",
    "Ahorro medio",
    "Por cobrar",
    "Potencial Credito",
    "Estado Deuda",
    "sub_estado_deuda",
    "estado_reparadora",
    "sub_estado_reparadora",
    "Mora_estructurado",
    "MORA_CREDITO",
    "Priority_level",
    "ultimo contacto",
    "CE"



]

# Validación: columnas faltantes
faltantes = [c for c in cols_cartera if c not in df_cartera.columns]
if faltantes:
    raise ValueError(f"Faltan estas columnas en df: {faltantes}")

# Nos quedamos solo con esas columnas
df_cartera = df_cartera[cols_cartera].copy()

# Opcional: eliminar duplicados por deuda (clave natural)
df_cartera = df_cartera.drop_duplicates(subset=["Id deuda"])

# Reset index limpio
df_cartera = df_cartera.reset_index(drop=True)

df_cartera.info()

<class 'pandas.DataFrame'>
RangeIndex: 41448 entries, 0 to 41447
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Referencia             41448 non-null  int64  
 1   Id deuda               41448 non-null  int64  
 2   Cedula                 41444 non-null  object 
 3   Nombre del cliente     41448 non-null  str    
 4   Negociador             41448 non-null  str    
 5   BANCOS_ESTANDAR        41448 non-null  str    
 6   Descuento              41447 non-null  float64
 7   D_BRAVO                41448 non-null  float64
 8   MORA                   41448 non-null  int64  
 9   Estructurable          41448 non-null  int64  
 10  Potencial              41446 non-null  float64
 11  Meses en el Programa   41448 non-null  int64  
 12  Tipo de Liquidacion    41448 non-null  str    
 13  Bucket                 41448 non-null  int64  
 14  Ahorro total           31235 non-null  float64
 15  Ahorro medio 

In [6]:
df_cartera

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORA,Estructurable,...,Potencial Credito,Estado Deuda,sub_estado_deuda,estado_reparadora,sub_estado_reparadora,Mora_estructurado,MORA_CREDITO,Priority_level,ultimo contacto,CE
0,3106653868,899359,3763762,Jorge Luis Camargo Fontalvo,Niyiret Julio Santos,Credivalores,0.267061,3996300.0,8,1,...,NaN,negotiation,negotiation,active,saving_paused,Sí,No,Prioridad 1,1,0.25
1,3106653868,899360,3763762,Jorge Luis Camargo Fontalvo,Niyiret Julio Santos,Serfinanza,0.248093,5763450.0,41,1,...,NaN,negotiation,negotiation,active,saving_paused,Sí,No,Prioridad 2,1,0.25
2,3173740082,1255027,1034308242,Nailen Josefina Bahamon Sandoval,Lina Maria Peña Gordillo,Banco Falabella,0.546325,13506762.0,10,0,...,NaN,negotiation,negotiation,active,saving_paused,No tiene estructurado,No,Prioridad 1,1,0.15
3,3224821336,1275933,1093755862,Diego Andres Delgado Galvis,Lina Maria Peña Gordillo,Banco Davivienda,0.279689,10257450.0,34,0,...,NaN,negotiation,negotiation,active,saving_paused,No tiene estructurado,No,Prioridad 2,1,0.15
4,3167086550,1297740,37826977,Luz Mery Quintero Blanco,Angie Natalia Borda Soler,Serfinanza,0.049949,8083900.0,33,0,...,NaN,negotiation,negotiation,active,saving_paused,No,No,Prioridad 1,0,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41443,3104785092,2024433,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,Gabriela Saavedra Latorre,Serfinanza,0.036529,3756500.0,12,0,...,NaN,new,NaN,active,saving_paused,No tiene estructurado,No,Prioridad 2,2,0.15
41444,3104785092,2024432,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,Gabriela Saavedra Latorre,BBVA Colombia,0.503870,948150.0,29,0,...,NaN,new,NaN,active,saving_paused,No tiene estructurado,No,Prioridad 2,2,0.15
41445,3104785092,2024430,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,Gabriela Saavedra Latorre,SisteCredito,0.161013,199500.0,11,0,...,NaN,new,NaN,active,saving_paused,No tiene estructurado,No,Prioridad 2,2,0.15
41446,3104785092,2024431,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,Gabriela Saavedra Latorre,SisteCredito,0.162588,741300.0,11,0,...,NaN,new,NaN,active,saving_paused,No tiene estructurado,No,Prioridad 2,2,0.15


In [7]:
# -*- coding: utf-8 -*-
import os
import json
import pandas as pd
import gspread
from datetime import datetime
from google.oauth2.service_account import Credentials

# =====================================
# 1. Cargar MI_JSON (Colab / GitHub)
# =====================================
def get_mi_json():
    try:
        # ---- Colab ----
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado en Colab userdata")
        print("Entorno detectado: Google Colab")
        return mi_json
    except Exception:
        # ---- GitHub Actions / local ----
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado como variable de entorno (GitHub/local)")
        if os.environ.get("GITHUB_ACTIONS") == "true":
            print("Entorno detectado: GitHub Actions")
        else:
            print("Entorno detectado: Local")
        return mi_json

mi_json = get_mi_json()
creds_dict = json.loads(mi_json)

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = Credentials.from_service_account_info(creds_dict, scopes=SCOPES)
gc = gspread.authorize(creds)

# =====================================
# 2. Definir año objetivo (regla 6 días)
# =====================================
today = datetime.today()

if today.month == 1 and today.day <= 6:
    target_year = today.year - 1
else:
    target_year = today.year

print(f"📅 Año objetivo detectado: {target_year}")

# =====================================
# 3. Abrir spreadsheet
# =====================================
SPREADSHEET_ID = "1O8OHuVhgwhLw8XYEBf1uBzLYrxQ45rPiZecHOnAa1Go"
sh = gc.open_by_key(SPREADSHEET_ID)

# =====================================
# 4. Buscar hoja por año en el nombre
# =====================================
worksheet_found = None
for ws in sh.worksheets():
    if str(target_year) in ws.title:
        worksheet_found = ws
        break

if worksheet_found is None:
    raise ValueError(
        f"No se encontró ninguna hoja cuyo nombre contenga el año {target_year}"
    )

print(f"✅ Hoja seleccionada: '{worksheet_found.title}'")

# =====================================
# 5. Leer a DataFrame
# =====================================
records = worksheet_found.get_all_records()

df_act = pd.DataFrame(records)

print("✅ df_act cargado correctamente")
print("Shape:", df_act.shape)
print(df_act.head())

Entorno detectado: GitHub Actions
📅 Año objetivo detectado: 2026


✅ Hoja seleccionada: 'act 2026'


✅ df_act cargado correctamente
Shape: (16301, 7)
                                        observations  \
0  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
1  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
2  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
3  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
4  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   

                               end  debt_id  payment_to_bank  \
0  vivian.rodriguez@gobravo.com.co  1898742  (314820000,COP)   
1  vivian.rodriguez@gobravo.com.co  1898737  (270710000,COP)   
2  vivian.rodriguez@gobravo.com.co  1898741  (313720000,COP)   
3  vivian.rodriguez@gobravo.com.co  1898738  (299530000,COP)   
4  vivian.rodriguez@gobravo.com.co  1898739  (304260000,COP)   

           inserted_at  bank_reference       Status_Act  
0  2026-01-02 13:10:20      3157363121  CONTRAPROPUESTA  
1  2026-01-02 13:10:53      3157363121  CONTRAPROPUESTA  
2  2026-01-02 13:09:48      3157363121  CONTRAPROPUESTA

In [8]:
df_act

,observations,end,debt_id,payment_to_bank,inserted_at,bank_reference,Status_Act
0,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898742,"(314820000,COP)",2026-01-02 13:10:20,3157363121,CONTRAPROPUESTA
1,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898737,"(270710000,COP)",2026-01-02 13:10:53,3157363121,CONTRAPROPUESTA
2,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898741,"(313720000,COP)",2026-01-02 13:09:48,3157363121,CONTRAPROPUESTA
3,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898738,"(299530000,COP)",2026-01-02 13:11:34,3157363121,CONTRAPROPUESTA
4,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898739,"(304260000,COP)",2026-01-02 13:12:05,3157363121,CONTRAPROPUESTA
...,...,...,...,...,...,...,...
16296,Alejo D//8079// 2:02 pm // Validamos chat de m...,norbey.duarte@gobravo.com.co,1922426,"(65000000,COP)",2026-02-09 19:05:03,3104106627,EN_CREDITO
16297,Jana|| 2:08 p. m. 9/02/2026 se adjunta acuerdo...,jana.lopez@gobravo.com.co,1932496,"(800034266,COP)",2026-02-09 19:12:31,3152331592,ACUERDO
16298,Yesseniaj/2:10pm/me comunico con Colpatria 601...,rosa.jimenez@gobravo.com.co,1962362,"(934865000,COP)",2026-02-09 19:11:30,3173761025,REVISAR
16299,2:15pm//Daniela Sanchez// deuda con velez moli...,maria.sarta@gobravo.com.co,1891368,"(1200000000,COP)",2026-02-09 19:19:07,3002117880,EN_CREDITO


In [9]:
import pandas as pd
import numpy as np
import re

df_act = df_act.copy()

s = df_act["inserted_at"]

# Pasar todo a string limpio (sin romper NaN)
s_str = s.astype("string").str.strip()

# Serie destino
dt = pd.Series(pd.NaT, index=df_act.index, dtype="datetime64[ns]")

# -----------------------------
# 1) Formato ISO: 2025-12-18 02:09:59 (o 2025-12-18T02:09:59Z)
# -----------------------------
mask_iso = s_str.str.match(r"^\d{4}-\d{2}-\d{2}", na=False)
iso_clean = (
    s_str[mask_iso]
    .str.replace("T", " ", regex=False)
    .str.replace("Z", "", regex=False)
)
dt.loc[mask_iso] = pd.to_datetime(iso_clean, errors="coerce")  # conserva hora

# -----------------------------
# 2) Formato con slash: 2/1/2025 (puede ser d/m/y o m/d/y)
# -----------------------------
mask_slash = s_str.str.contains(r"/", na=False) & (~mask_iso)

# extraer día/mes/año como números
parts = s_str[mask_slash].str.extract(r"^\s*(\d{1,2})/(\d{1,2})/(\d{4})\s*$")
a = pd.to_numeric(parts[0], errors="coerce")  # primera parte
b = pd.to_numeric(parts[1], errors="coerce")  # segunda parte

# reglas para decidir dayfirst vs monthfirst
mask_dayfirst = (a > 12) & (b <= 12)
mask_monthfirst = (b > 12) & (a <= 12)
mask_ambigua = ~(mask_dayfirst | mask_monthfirst)

idx_slash = parts.index

# dayfirst seguro
idx_day = idx_slash[mask_dayfirst.fillna(False)]
dt.loc[idx_day] = pd.to_datetime(s_str.loc[idx_day], errors="coerce", dayfirst=True)

# monthfirst seguro
idx_mon = idx_slash[mask_monthfirst.fillna(False)]
dt.loc[idx_mon] = pd.to_datetime(s_str.loc[idx_mon], errors="coerce", dayfirst=False)

# ambigua -> por defecto dayfirst=True (tu estándar)
idx_amb = idx_slash[mask_ambigua.fillna(True)]
tmp = pd.to_datetime(s_str.loc[idx_amb], errors="coerce", dayfirst=True)

# si alguna ambigua falla, reintenta monthfirst
mask_fail = tmp.isna()
if mask_fail.any():
    tmp.loc[mask_fail] = pd.to_datetime(s_str.loc[idx_amb[mask_fail]], errors="coerce", dayfirst=False)

dt.loc[idx_amb] = tmp

# -----------------------------
# 3) Guardar resultado final
# -----------------------------
df_act["inserted_at"] = dt

# (Opcional) Ver cuántos quedaron NaT
print("NaT en inserted_at:", df_act["inserted_at"].isna().sum())
print("dtype:", df_act["inserted_at"].dtype)

NaT en inserted_at: 0
dtype: datetime64[ns]


In [10]:
import pandas as pd
import numpy as np
import re

df_act["payment_to_bank"] = (
    df_act["payment_to_bank"]
    .astype(str)
    .str.extract(r"\(?\s*([\d]+)\s*,?\s*COP?\s*\)?", expand=False)
    .astype(float)
)

In [11]:
df_act['payment_to_bank'] = df_act['payment_to_bank']/100

In [12]:
df_act = df_act.rename(columns={"Status_Act": "CATEGORIA_PRED"})

In [13]:
df_act

,observations,end,debt_id,payment_to_bank,inserted_at,bank_reference,CATEGORIA_PRED
0,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898742,3148200.00,2026-01-02 13:10:20,3157363121,CONTRAPROPUESTA
1,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898737,2707100.00,2026-01-02 13:10:53,3157363121,CONTRAPROPUESTA
2,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898741,3137200.00,2026-01-02 13:09:48,3157363121,CONTRAPROPUESTA
3,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898738,2995300.00,2026-01-02 13:11:34,3157363121,CONTRAPROPUESTA
4,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898739,3042600.00,2026-01-02 13:12:05,3157363121,CONTRAPROPUESTA
...,...,...,...,...,...,...,...
16296,Alejo D//8079// 2:02 pm // Validamos chat de m...,norbey.duarte@gobravo.com.co,1922426,650000.00,2026-02-09 19:05:03,3104106627,EN_CREDITO
16297,Jana|| 2:08 p. m. 9/02/2026 se adjunta acuerdo...,jana.lopez@gobravo.com.co,1932496,8000342.66,2026-02-09 19:12:31,3152331592,ACUERDO
16298,Yesseniaj/2:10pm/me comunico con Colpatria 601...,rosa.jimenez@gobravo.com.co,1962362,9348650.00,2026-02-09 19:11:30,3173761025,REVISAR
16299,2:15pm//Daniela Sanchez// deuda con velez moli...,maria.sarta@gobravo.com.co,1891368,12000000.00,2026-02-09 19:19:07,3002117880,EN_CREDITO


In [14]:
import pandas as pd

# ✅ Compatibilidad ZoneInfo: en Py<3.9 (algunos entornos) usa backports
try:
    from zoneinfo import ZoneInfo
except ImportError:  # pragma: no cover
    from backports.zoneinfo import ZoneInfo  # type: ignore


def construir_timeline_mes(
    df_cartera: pd.DataFrame,
    df_act: pd.DataFrame,
    ref_date=None,
    tz: str = "America/Bogota",
) -> pd.DataFrame:
    """
    Timeline por deuda:
    - Última observación antes del mes
    - Todas las observaciones del mes actual
    Requiere que df_act ya tenga columna 'CATEGORIA_PRED' si quieres usarla.
    """

    # =========================
    # 0) Copias defensivas
    # =========================
    df_c = df_cartera.copy()
    df_a = df_act.copy()

    # =========================
    # 1) Fecha de referencia
    # =========================
    tzinfo = ZoneInfo(tz)

    if ref_date is None:
        ref_date = pd.Timestamp.now(tzinfo)
    else:
        ref_date = pd.Timestamp(ref_date)
        if ref_date.tzinfo is None:
            ref_date = ref_date.tz_localize(tzinfo)
        else:
            ref_date = ref_date.tz_convert(tzinfo)

    month_start = ref_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    next_month_start = month_start + pd.offsets.MonthBegin(1)

    # =========================
    # 2) Normalizar fechas
    # =========================
    df_a["inserted_at"] = pd.to_datetime(df_a["inserted_at"], errors="coerce")

    if getattr(df_a["inserted_at"].dt, "tz", None) is None:
        df_a["inserted_at"] = df_a["inserted_at"].dt.tz_localize(
            tzinfo,
            nonexistent="shift_forward",
            ambiguous="NaT",
        )
    else:
        df_a["inserted_at"] = df_a["inserted_at"].dt.tz_convert(tzinfo)

    # =========================
    # 3) Separar eventos
    # =========================
    mask_mes = (df_a["inserted_at"] >= month_start) & (df_a["inserted_at"] < next_month_start)

    df_mes = df_a.loc[mask_mes].copy()
    df_prev = df_a.loc[df_a["inserted_at"] < month_start].copy()

    # =========================
    # 4) Última antes del mes
    # =========================
    df_prev = df_prev.sort_values(["debt_id", "inserted_at"])
    ultima_prev = df_prev.groupby("debt_id", as_index=False).tail(1)
    ultima_prev["tipo_fila"] = "ultima_antes_mes"

    # =========================
    # 5) Todas las del mes
    # =========================
    df_mes = df_mes.sort_values(["debt_id", "inserted_at"])
    df_mes["tipo_fila"] = "mes_actual"

    # =========================
    # 6) Unir eventos
    # =========================
    eventos = pd.concat([ultima_prev, df_mes], ignore_index=True)

    # Columnas de eventos que quieres traer sí o sí
    cols_eventos = [
        "bank_reference",
        "debt_id",
        "inserted_at",
        "end",
        "payment_to_bank",
        "CATEGORIA_PRED",
        "observations",
        "tipo_fila",
    ]
    for c in cols_eventos:
        if c not in eventos.columns:
            eventos[c] = pd.NA
    eventos = eventos[cols_eventos]

    # =========================
    # 7) Merge con cartera
    # =========================
    df_timeline = df_c.merge(
        eventos,
        left_on=["Referencia", "Id deuda"],
        right_on=["bank_reference", "debt_id"],
        how="left",
    ).drop(columns=["bank_reference", "debt_id"])

    # =========================
    # 8) Garantizar columnas requeridas en el output
    # =========================
    cols_cartera_requeridas = [
        "Referencia",
        "Id deuda",
        "Cedula",
        "Nombre del cliente",
        "Negociador",
        "BANCOS_ESTANDAR",
        "Descuento",
        "D_BRAVO",
        "MORA",
        "Estructurable",
        "Potencial",
        "Meses en el Programa",
        "Tipo de Liquidacion",
        "Bucket",
        "Ahorro total",
        "Ahorro medio",
        "Por cobrar",
        "Potencial Credito",
        "Estado Deuda",
        "sub_estado_deuda",
        "estado_reparadora",
        "sub_estado_reparadora",
        "Mora_estructurado",
        "MORA_CREDITO",
        "Priority_level",
        "ultimo contacto",
        "fecha mensaje",
        "CE"
    ]

    # Si alguna no existe, créala en NA para evitar KeyError
    for c in cols_cartera_requeridas:
        if c not in df_timeline.columns:
            df_timeline[c] = pd.NA

    # Orden sugerido: primero tus columnas de cartera, luego las de eventos (y luego cualquier extra que exista)
    cols_eventos_out = ["inserted_at", "end", "payment_to_bank", "CATEGORIA_PRED", "observations", "tipo_fila"]
    extras = [c for c in df_timeline.columns if c not in (cols_cartera_requeridas + cols_eventos_out)]

    df_timeline = df_timeline[cols_cartera_requeridas + cols_eventos_out + extras]

    # =========================
    # 9) Orden final
    # =========================
    df_timeline = df_timeline.sort_values(
        ["Id deuda", "inserted_at"],
        na_position="first"
    ).reset_index(drop=True)

    return df_timeline


# ===== EJECUCIÓN =====
df_timeline = construir_timeline_mes(df_cartera, df_act)
df_timeline.head(20)

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORA,Estructurable,...,Priority_level,ultimo contacto,fecha mensaje,CE,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,90,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,91,1,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,92,1,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,90,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,102,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
5,3504614613,6828,-,ANTONIO MARCOS TAMAYO TORRES,Katherine Marulanda Bernal,Banco Davivienda,0.034976,11528000.0,96,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
6,3504614613,6829,-,ANTONIO MARCOS TAMAYO TORRES,Katherine Marulanda Bernal,Itaú,0.450122,3768000.0,96,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
7,3153844365,6959,-,EDINSON JIMENEZ RIZO,Hector Elian Lacera Vega,Banco AV Villas,0.467291,14766000.0,90,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
8,3153844365,6960,-,EDINSON JIMENEZ RIZO,Hector Elian Lacera Vega,BBVA Colombia,0.448483,8412000.0,97,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
9,3183411742,7204,-,JONATHAN CAMILO GIRALDO GOMEZ,Karen Sofia Parra Gaspar,Banco Davivienda,0.038111,37560000.0,96,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN


In [15]:
df_timeline

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORA,Estructurable,...,Priority_level,ultimo contacto,fecha mensaje,CE,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,90,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,91,1,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,92,1,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,90,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,102,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44397,3206721655,2121952,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164459,274220.0,8,1,...,Prioridad 2,0,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
44398,3206721655,2121953,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164761,370308.0,8,1,...,Prioridad 2,0,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
44399,3206721655,2121954,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.166528,434983.0,8,1,...,Prioridad 2,0,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN
44400,3108495298,2122000,1052379703,Edwin Augusto Medina Gallo,Julian Andres Ossa Bohorquez,Nu,0.085922,6615000.0,3,0,...,Prioridad 2,6,<NA>,0.15,NaT,NaN,NaN,NaN,NaN,NaN


In [16]:
# -*- coding: utf-8 -*-
import os
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# =========================
# CONFIG
# =========================
SPREADSHEET_ID = "1H3sYEtkeu47POnu8xZMaMtID1Vj53YIcWblWeZ8d0rc"
GID = "1033250632"

# =========================
# 1) Leer MI_JSON según entorno (Colab vs GitHub/local)
# =========================
def get_mi_json():
    # --- Colab ---
    try:
        from google.colab import userdata  # solo existe en Colab
        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado en Colab userdata.")
        print("Entorno detectado: Google Colab")
        return mi_json
    except Exception:
        # --- GitHub Actions / local ---
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado como variable de entorno (GitHub/local).")
        if os.environ.get("GITHUB_ACTIONS") == "true":
            print("Entorno detectado: GitHub Actions")
        else:
            print("Entorno detectado: Local")
        return mi_json

mi_json = get_mi_json()
info = json.loads(mi_json)

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly",
]
creds = Credentials.from_service_account_info(info, scopes=SCOPES)
gc = gspread.authorize(creds)

sh = gc.open_by_key(SPREADSHEET_ID)

# =========================
# 2) Resolver el nombre de la hoja a partir del gid
# =========================
meta = sh.fetch_sheet_metadata()
sheet_title = None
for s in meta.get("sheets", []):
    props = s.get("properties", {})
    if str(props.get("sheetId")) == str(GID):
        sheet_title = props.get("title")
        break

if not sheet_title:
    raise ValueError(f"No encontré ninguna hoja con gid={GID}. Revisa el link.")

ws = sh.worksheet(sheet_title)

# =========================
# 3) Leer a DataFrame
# =========================
values = ws.get_all_values()

if not values or len(values) < 2:
    df_liq = pd.DataFrame()
else:
    headers = values[0]
    rows = values[1:]
    df_liq = pd.DataFrame(rows, columns=headers)

print("✅ Hoja leída:", sheet_title)
print("Shape:", df_liq.shape)
print(df_liq.head(20))

Entorno detectado: GitHub Actions


✅ Hoja leída: BD del mes
Shape: (2657, 27)
   Pago a banco Deuda Resuelve  Fecha de Liquidación  Comisión  \
0       2075000         8520741            3/02/2026    966861   
1        993400         3735449            3/02/2026    548410   
2        500000         1056300            3/02/2026     83445   
3        600000         2245950            3/02/2026    246893   
4        603768          648900            3/02/2026      6770   
5        218453          265650            3/02/2026      7080   
6        133421          141750            4/02/2026      8041   
7        323510          339150            4/02/2026             
8        612964          642600            4/02/2026             
9      37000000       238385700            3/02/2026  26461681   
10      2550000        17704050            3/02/2026   3030810   
11       800000        28812000            4/02/2026   2881200   
12      1500000        10975200            4/02/2026   1895040   
13      1811000        13273700  

In [17]:
df_liq

,Pago a banco,Deuda Resuelve,Fecha de Liquidación,Comisión,Negociador,Portafolio,Deuda Berex,Referencia,Nombre cliente,Banco,...,Tipo EST,Dia operativo,Check,Lider,Casa de cobro,,,,,
0,2075000,8520741,3/02/2026,966861,Rosa Yessenia Jimenez Lara,NO,1938575,3223950956,Luisa Fernanda Rios Mora,Éxito,...,comisión,9,,Jordann Dario Barbosa Villanueva,0,TRUE,"15,00%",,1681103,comisión
1,993400,3735449,3/02/2026,548410,Maria Alejandra Bejarano Tumay,NO,1378109,3103075029,Diana Alexandra Pena Lopez,Falabella,...,,9,,Jordann Dario Barbosa Villanueva,0,TRUE,"20,00%",,1726199,comisión
2,500000,1056300,3/02/2026,83445,Diego Alejandro Sanchez Fonseca,NO,1915657,3196049552,Miguel Hernando Ramirez Nieto,Éxito,...,,9,,Jordann Dario Barbosa Villanueva,0,TRUE,"15,00%",,1773378,comisión
3,600000,2245950,3/02/2026,246893,Maria Daniela Sarta Sanchez,NO,1968166,3057662390,Andrea Teresa Del Pilar Fajardo Restrepo,Alkosto,...,,9,,Jordann Dario Barbosa Villanueva,0,TRUE,"15,00%",,1203220,comisión
4,603768,648900,3/02/2026,6770,Diego Alejandro Sanchez Fonseca,NO,1974043,3017994883,Cristian Camilo Castillo Cadena,FGA,...,,9,,Jordann Dario Barbosa Villanueva,0,TRUE,"15,00%",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2652,,,,,,,,,,,...,,,,,0,,,,,
2653,,,,,,,,,,,...,,,,,0,,,,,
2654,,,,,,,,,,,...,,,,,0,,,,,
2655,,,,,,,,,,,...,,,,,0,,,,,


In [18]:
pip install fuzzywuzzy[speedup]

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)


  Using cached python_levenshtein-0.27.3-py3-none-any.whl.metadata (3.9 kB)


  Using cached levenshtein-0.27.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.7 kB)


Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
Using cached python_levenshtein-0.27.3-py3-none-any.whl (9.5 kB)
Using cached levenshtein-0.27.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (153 kB)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [python-levenshtein]


Note: you may need to restart the kernel to use updated packages.


In [19]:
from rapidfuzz import process, fuzz
import pandas as pd
import re

nombres_correctos = [
    'Bancolombia', 'Banco Falabella', 'Banco de Bogotá', 'Banco Davivienda',
    'Scotiabank Colpatria', 'BBVA Colombia', 'SisteCredito', 'Banco AV Villas',
    'Banco de Occidente', 'Alkomprar', 'Tuya', 'Codensa', 'Itaú',
    'Serfinanza', 'Credivalores', 'Banco Popular', 'Rappipay',
    'Banco Finandina', 'Banco Caja Social', 'Covinoc', 'Bancoomeva',
    'Rapicredit', 'Credijamar', 'Flamingo', 'Zinobe', 'Bancamia',
    'Refinancia', 'Compensar', 'Pichincha', 'Colsubsidio', 'Mundo Mujer',
    'Agaval', 'Sistemcobro', 'Serlefin', 'Rappicard', 'Fincomercio',
    'GRUPO JURIDICO DEUDU', 'AECSA', 'Sufi', 'Comultrasan', 'Fundación',
    'Cobrando', 'Aslegal', 'Coltefinanciera', 'Reestructura', 'Nu',
    'Juancho te Presta', 'Muebles', 'JOHN', 'GNB Sudameris', 'Confiar',
    'Baninca', 'Systemgroup', 'Efecty'
]

# Bancos que se usan con el prefijo "banco" completo como referencia
bancos_excepciones = ['Banco de Bogotá', 'Bancolombia', 'Bancoomeva']

# ==============================
# 2. Sinónimos / patrones por banco (texto original, sin limpiar)
# ==============================
PATRONES_POR_BANCO = {
    'Bancolombia': [
        'bancolombia', 'contento bancolombia', 'qnt bancolombia', 'sufi'
    ],
    'Banco Davivienda': [
        'davivienda', 'intercredito davivienda', 'gestiones profesionales davivienda',
        'inversionistas estratégicos davivienda', 'inversionistas estrategicos davivienda',
        'management davivienda', 'davivienda cobrado sas', 'davivienda cobrando sas',
        'deudu davivienda', 'qnt davivienda'
    ],
    'BBVA Colombia': [
        'bbva', 'cobrando bbva', 'beta bbva', 'cobranzas beta origen: bbva',
        'aecsa bbva', 'grupo juridico bbva', 'grupo jurídico bbva', 'qnt bbva'
    ],
    'Banco Falabella': [
        'falabella', 'bfalabella', 'bancofalab', 'bancofalab citisumma',
        'banco falabella casa de cobro', 'eyc falabella',
        'cobrando falabella', 'acr logros f ori falabella',
        'deudu falabella', 'deudo falabella', 'citisumma falabella',
        'logros factoring falabella'
    ],
    'Tuya': [
        ' tuya', 'viva-tuya', 'viva tuya', 'qnt tuya', 'aecsa tuya',
        'tuya contacto soluciones', 'tuya s.a contactosol', 'tuya s a contactosol',
        'tuya contacto soluciones', 'qnt tuya',
        'éxito', 'exito', 'carulla', 'Alkosto', 'Corbeta'
    ],
    'Scotiabank Colpatria': [
        'scotiabank', 'skotiabank', 'colpatria', 'peruzzi skotiabank colpatria',
        'serlefin colpatria', 'adamantine scotiabank', 'gc andino colpatria',
        'scotiabank citibank', 'qnt colpatria', 'crc colpatria',
        'grupo consulto colpatria', 'grupo consultor andino colpatria',
        'gr.consulto colpatria', 'Codensa'
    ],
    'Banco de Bogotá': [
        'banco de bogota', 'banco de bogotá', 'qnt bogota', 'qnt bogotá',
        'crear país banco de bogotá', 'crear pais banco de bogota'
    ],
    'Banco de Occidente': [
        'banco de occidente', 'qnt banco de occidente', 'deudu-banco de occidente',
        'deudu banco de occidente'
    ],
    'Banco Popular': [
        'banco popular', 'banco popular casa de cobro', 'banco popular contactosol',
        'banco popular contactosolsas', 'banco popular citisumma',
        'banco popular-adcore', 'deudu banco popular', 'peruzzicol bcopopular'
    ],
    'Banco AV Villas': [
        'av villas', 'banco av villas', 'grupo consultor andino av villas',
        'grupo juridico av villas', 'grupo jurídico av villas',
        'ae csa av villas', 'aecsa av villas', 'crear país banco av villas',
        'crear pais banco av villas', 'deudu av villas', 'qnt av villas'
    ],
    'Banco Caja Social': [
        'banco caja social', 'caja social', 'pic caja social'
    ],
    'Bancoomeva': [
        'banco coomeva', 'bancoomeva', 'coomeva'
    ],
    'Bancamia': [
        'bancamia', 'bancamia s.a', 'bancamia s a'
    ],
    'Mundo Mujer': [
        'banco mundo mujer', 'fundacion de la mujer', 'fundación de la mujer'
    ],
    'SisteCredito': [
        'sistecredito', 'sistecrédito'
    ],
    'Covinoc': ['covinoc'],
    'Compensar': ['compensar'],
    'Pichincha': ['pichincha', 'pichincha educativo'],
    'Agaval': ['agaval'],
    'Banco Finandina': [
        'finandina', 'finandina incomercio', 'finandina incomercio'
    ],
    'Fincomercio': ['fincomercio'],
    'Serfinanza': [
        'serfinanza', 'serfinansa', 'serfinanza contactosol',
        'contacto solucion serfinanza', 'contacto solución serfinanza'
    ],
    'Credijamar': [
        'credijamar', 'muebles jamar'
    ],
    'Juancho te Presta': ['juancho te presta'],
    'Rapicredit': ['rapicredit'],
    'Zinobe': ['zinobe'],
    'Coltefinanciera': ['coltefinanciera'],
    'Sistemcobro': ['sistemcobro'],
    'Systemgroup': ['systemgroup'],
    'Baninca': ['baninca'],
    'GNB Sudameris': ['gnb sudameris'],
    'Confiar': ['confiar'],
    'AECSA': ['aecsa'],
    'Comultrasan': ['comultrasan'],
    'Nu': ['nu bank', 'nubank', 'logros factoring nubank'],
    'Rappi':['Rappicard', 'Rappipay'],
    'Lulo Bank': ['LuloBank', 'Lulo Banck'],
    'Banco union': ['QNT GIROS&FINANZAS']

}

# ==============================
# 3. Alias manuales sobre texto LIMPIO
# ==============================
alias_manuales = {
    'bfalabella': 'Banco Falabella',
    'contactosol': 'Banco Falabella',
    'qnt itau': 'Itaú',
    'itau helm': 'Itaú',
    'itau corpbanca': 'Itaú',
    'viva tuya': 'Tuya',
    'carulla': 'Tuya',
    'sufi': 'Bancolombia',
    'adamantine scotiabank': 'Scotiabank Colpatria',
    'gc andino colpatria': 'Scotiabank Colpatria',
    'beta bbva': 'BBVA Colombia',
    'banco popular': 'Banco Popular',
    'qnt bogota': 'Banco de Bogotá',
    'banco caja social': 'Banco Caja Social',
    'banco av villas': 'Banco AV Villas',
    'banco davivienda': 'Banco Davivienda',
    'bancofalab citisumma': 'Banco Falabella',
    'exito': 'Tuya',
    'éxito': 'Tuya',
    'Alkosto': 'Tuya',
    'Corbeta': 'Tuya',
    'Rappipay': 'Rappi',
    'Rappicard': 'Rappi',
    'Lulo Banck': 'Lulo Bank',
    'LuloBank': 'Lulo Bank',
    'QNT GIROS&FINANZAS': 'Banco Union',
    'Codensa': 'Scotiabank Colpatria'


}

# ==============================
# 4. Función de limpieza
# ==============================
def limpiar_texto(texto):
    """
    Limpia y normaliza el texto: minúsculas, elimina caracteres no alfabéticos
    (excepto acentos y ñ/ü), elimina palabras irrelevantes y quita espacios dobles.
    """
    texto = str(texto).lower()

    # Dejar solo letras, acentos, ñ, ü y espacios
    texto = re.sub(r'[^a-záéíóúñü\s]', ' ', texto)

    # Eliminar palabras "de relleno" frecuentes en reparadoras / BPO
    texto = re.sub(
        r'\b('
        r'grupo|juridico|jurídico|sas|sa|s a|ltda|suma|financiera|'
        r'contactosol|contacto|solucion|soluciones|citisumma|'
        r'cobrando|cobranzas|adcore|logros|factoring|origen|origem|'
        r'gestiones|gestion|profesionales|bpo|inversionistas|'
        r'estrategicos|estratégicos|casa|de|cobro|servicios|'
        r'creditos|credito|abogados|asociados|'
        r'outsourcing|risk|patrimonio|autonomo|autónomo|central|'
        r'inversiones|valora|punto|com|puntocom|activos|'
        r'recuperacion|recuperación|financiera|financiero|'
        r'asesores|asociados|gest|prof|eyc|gca|summa'
        r')\b',
        '',
        texto
    )

    # Espacios múltiples -> uno solo
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# ==============================
# 5. Claves de referencia para fuzzy
# ==============================
claves_referencia = {
    (n.lower() if n in bancos_excepciones else re.sub(r'^banco\s*', '', n.lower())): n
    for n in nombres_correctos
}

# ==============================
# 6. Mapeo por patrón (texto original)
# ==============================
def mapear_por_patron(nombre_incorrecto):
    """
    Intenta identificar el banco a partir del texto ORIGINAL (sin limpiar),
    buscando los patrones definidos en PATRONES_POR_BANCO.
    """
    if pd.isna(nombre_incorrecto):
        return None

    texto = str(nombre_incorrecto).lower()

    for banco_estandar, patrones in PATRONES_POR_BANCO.items():
        for patron in patrones:
            if patron in texto:
                return banco_estandar

    return None

# ==============================
# 7. Función principal de corrección
# ==============================
def corregir_nombre(nombre_incorrecto):
    """
    Corrige un nombre de banco usando 3 capas:
    1. Mapeo por patrón sobre el texto original.
    2. Limpieza de texto + alias manuales.
    3. Fuzzy matching contra claves_referencia.
    """
    # Si es NaN/None, lo dejamos igual
    if pd.isna(nombre_incorrecto):
        return nombre_incorrecto

    # 1. Intentar primero con patrones evidentes en el texto original
    banco_patron = mapear_por_patron(nombre_incorrecto)
    if banco_patron is not None:
        return banco_patron

    # 2. Limpiar texto
    limpio = limpiar_texto(nombre_incorrecto)

    # Si después de limpiar no queda nada, devolvemos el original
    if limpio == '':
        return nombre_incorrecto

    # 3. Alias manuales sobre texto limpio
    for alias, banco_estandar in alias_manuales.items():
        if alias in limpio:
            return banco_estandar

    # 4. Fuzzy matching con RapidFuzz
    mejor_match, score, _ = process.extractOne(
        limpio,
        claves_referencia.keys(),
        scorer=fuzz.token_set_ratio
    )

    # Si la similitud es alta, usamos el banco estándar; si no, dejamos el texto original
    return claves_referencia[mejor_match] if score > 70 else nombre_incorrecto

# ==============================
# 8. Aplicar al DataFrame
# ==============================

# Crear nueva columna estandarizada en reparadoras_df
df_liq['BANCOS_ESTANDAR'] = df_liq['Banco'].apply(corregir_nombre)

In [20]:
import pandas as pd
import numpy as np

# ✅ ZoneInfo compatible con Colab y GitHub (Py < 3.9)
try:
    from zoneinfo import ZoneInfo
except ImportError:  # pragma: no cover
    from backports.zoneinfo import ZoneInfo  # type: ignore


def _to_float_money(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan

    s = s.replace("$", "").replace("COP", "").replace("cop", "").replace(" ", "")

    if "," in s and "." in s:
        if s.rfind(",") > s.rfind("."):
            s = s.replace(".", "")
            s = s.replace(",", ".")
        else:
            s = s.replace(",", "")
    else:
        if "," in s:
            if s.count(",") > 1:
                s = s.replace(",", "")
            else:
                tail = s.split(",")[-1]
                s = s.replace(",", ".") if len(tail) in (1, 2) else s.replace(",", "")
        if "." in s:
            if s.count(".") > 1:
                s = s.replace(".", "")
            else:
                tail = s.split(".")[-1]
                if len(tail) not in (1, 2):
                    s = s.replace(".", "")

    try:
        return float(s)
    except Exception:
        return np.nan


def _to_bogota_datetime_ddmmyyyy(series, tz="America/Bogota"):
    tzinfo = ZoneInfo(tz)
    dt = pd.to_datetime(series, errors="coerce", dayfirst=True)
    if getattr(dt.dt, "tz", None) is None:
        dt = dt.dt.tz_localize(tzinfo, nonexistent="shift_forward", ambiguous="NaT")
    else:
        dt = dt.dt.tz_convert(tzinfo)
    return dt


def _modo(s: pd.Series):
    s = s.dropna().astype(str)
    if s.empty:
        return np.nan
    return s.value_counts().index[0]


def agregar_liquidaciones_al_timeline_con_fallback(
    df_timeline: pd.DataFrame,
    df_liq: pd.DataFrame,
) -> pd.DataFrame:
    df_tl = df_timeline.copy()
    df_l = df_liq.copy()

    # asegurar columnas nuevas
    for c in ["Negociador liquidacion", "Por?"]:
        if c not in df_tl.columns:
            df_tl[c] = np.nan

    # normalizar llaves
    df_l["Id deuda"] = pd.to_numeric(df_l["Deuda Berex"], errors="coerce").astype("Int64")
    df_l["Referencia"] = pd.to_numeric(df_l["Referencia"], errors="coerce").astype("Int64")

    # fecha dd/mm/yyyy -> datetime Bogotá
    df_l["inserted_at"] = _to_bogota_datetime_ddmmyyyy(df_l["Fecha de Liquidación"])

    # pago a banco float
    df_l["payment_to_bank"] = df_l["Pago a banco"].apply(_to_float_money).astype(float)

    # mapas desde timeline para fallback  ✅ (aquí solo añadimos tus 4 columnas)
    base_by_debt = (
        df_tl[
            [
                "Id deuda",
                "Referencia",
                "BANCOS_ESTANDAR",
                "Descuento",
                "D_BRAVO",
                "Tipo de Liquidacion",
                "Ahorro total",
                "Ahorro medio",
                "Potencial",
                "Potencial Credito",
            ]
        ]
        .dropna(subset=["Id deuda"])
        .drop_duplicates(subset=["Id deuda"])
        .set_index("Id deuda")
    )

    map_ref_by_debt = base_by_debt["Referencia"].to_dict()
    map_banco_by_debt = base_by_debt["BANCOS_ESTANDAR"].to_dict()
    map_desc_by_debt = base_by_debt["Descuento"].to_dict()
    map_deuda_by_debt = base_by_debt["D_BRAVO"].to_dict()
    map_tipol_by_debt = base_by_debt["Tipo de Liquidacion"].to_dict()

    # ✅ nuevos mapas
    map_ahorro_total_by_debt = base_by_debt["Ahorro total"].to_dict()
    map_ahorro_medio_by_debt = base_by_debt["Ahorro medio"].to_dict()
    map_potencial_by_debt = base_by_debt["Potencial"].to_dict()
    map_pot_credito_by_debt = base_by_debt["Potencial Credito"].to_dict()

    # mapa negociador por referencia
    map_neg_ref = (
        df_tl[["Referencia", "Negociador"]]
        .dropna(subset=["Referencia", "Negociador"])
        .groupby("Referencia")["Negociador"]
        .apply(_modo)
        .to_dict()
    )

    banco_liq = df_l["BANCO_ESTANDAR"] if "BANCO_ESTANDAR" in df_l.columns else pd.Series(np.nan, index=df_l.index)
    deuda_res_liq = df_l["D_BRAVO"] if "D_BRAVO" in df_l.columns else pd.Series(np.nan, index=df_l.index)

    if "descuento" in df_l.columns:
        desc_liq = pd.to_numeric(df_l["descuento"], errors="coerce")
    else:
        desc_liq = pd.Series(np.nan, index=df_l.index)

    tipo_liq_liq = df_l["Tipo de liquidacion"].astype(object)

    filas_liq = pd.DataFrame({
        "Referencia": df_l["Referencia"],
        "Id deuda": df_l["Id deuda"],
        "Negociador": np.nan,
        "BANCOS_ESTANDAR": banco_liq.astype(object),
        "Descuento": desc_liq.astype(float),
        "D_BRAVO": deuda_res_liq,
        "Tipo de Liquidacion": tipo_liq_liq,

        # ✅ nuevas columnas en la fila Liquidado (arrancan nulas)
        "Ahorro total": np.nan,
        "Ahorro medio": np.nan,
        "Potencial": np.nan,
        "Potencial Credito": np.nan,

        "inserted_at": df_l["inserted_at"],
        "end": np.nan,
        "payment_to_bank": df_l["payment_to_bank"],
        "CATEGORIA_PRED": "Liquidado",
        "observations": np.nan,
        "tipo_fila": "Liquidación",
        "Negociador liquidacion": df_l["Negociador"].astype(str),
        "Por?": df_l["Tipo de liquidacion"].astype(str),
    })

    filas_liq = (
        filas_liq.dropna(subset=["Id deuda"])
        .sort_values(["Id deuda", "inserted_at"])
        .groupby("Id deuda", as_index=False)
        .tail(1)
    )

    filas_liq["Referencia"] = filas_liq["Referencia"].fillna(filas_liq["Id deuda"].map(map_ref_by_debt))
    filas_liq["Negociador"] = filas_liq["Referencia"].map(map_neg_ref)

    filas_liq["BANCOS_ESTANDAR"] = filas_liq["BANCOS_ESTANDAR"].fillna(filas_liq["Id deuda"].map(map_banco_by_debt))
    filas_liq["Descuento"] = filas_liq["Descuento"].fillna(filas_liq["Id deuda"].map(map_desc_by_debt))

    filas_liq["D_BRAVO"] = pd.to_numeric(filas_liq["D_BRAVO"], errors="coerce")
    filas_liq["D_BRAVO"] = filas_liq["D_BRAVO"].fillna(filas_liq["Id deuda"].map(map_deuda_by_debt))

    filas_liq["Tipo de Liquidacion"] = filas_liq["Tipo de Liquidacion"].replace("", np.nan)
    filas_liq["Tipo de Liquidacion"] = filas_liq["Tipo de Liquidacion"].fillna(
        filas_liq["Id deuda"].map(map_tipol_by_debt)
    )

    # ✅ fallback nuevo: igualito a lo demás
    filas_liq["Ahorro total"] = filas_liq["Ahorro total"].fillna(filas_liq["Id deuda"].map(map_ahorro_total_by_debt))
    filas_liq["Ahorro medio"] = filas_liq["Ahorro medio"].fillna(filas_liq["Id deuda"].map(map_ahorro_medio_by_debt))
    filas_liq["Potencial"] = filas_liq["Potencial"].fillna(filas_liq["Id deuda"].map(map_potencial_by_debt))
    filas_liq["Potencial Credito"] = filas_liq["Potencial Credito"].fillna(
        filas_liq["Id deuda"].map(map_pot_credito_by_debt)
    )

    filas_liq["Referencia"] = filas_liq["Referencia"].astype("Int64")
    filas_liq["Id deuda"] = filas_liq["Id deuda"].astype("Int64")
    filas_liq["Descuento"] = pd.to_numeric(filas_liq["Descuento"], errors="coerce").astype(float)
    filas_liq["D_BRAVO"] = pd.to_numeric(filas_liq["D_BRAVO"], errors="coerce").astype(float)
    filas_liq["payment_to_bank"] = pd.to_numeric(filas_liq["payment_to_bank"], errors="coerce").astype(float)
    filas_liq["inserted_at"] = _to_bogota_datetime_ddmmyyyy(filas_liq["inserted_at"])

    for c in df_tl.columns:
        if c not in filas_liq.columns:
            filas_liq[c] = np.nan
    for c in filas_liq.columns:
        if c not in df_tl.columns:
            df_tl[c] = np.nan

    filas_liq = filas_liq[df_tl.columns]

    out = (
        pd.concat([df_tl, filas_liq], ignore_index=True)
        .sort_values(["Id deuda", "inserted_at"], na_position="first")
        .reset_index(drop=True)
    )

    return out


# ===== USO =====
df_timeline_final = agregar_liquidaciones_al_timeline_con_fallback(df_timeline, df_liq)
df_timeline_final.info()

<class 'pandas.DataFrame'>
RangeIndex: 44469 entries, 0 to 44468
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype                         
---  ------                  --------------  -----                         
 0   Referencia              44469 non-null  Int64                         
 1   Id deuda                44469 non-null  Int64                         
 2   Cedula                  44398 non-null  object                        
 3   Nombre del cliente      44402 non-null  object                        
 4   Negociador              44467 non-null  str                           
 5   BANCOS_ESTANDAR         44427 non-null  object                        
 6   Descuento               44426 non-null  float64                       
 7   D_BRAVO                 44427 non-null  float64                       
 8   MORA                    44402 non-null  float64                       
 9   Estructurable           44402 non-null  float64              

In [21]:
import numpy as np

df_timeline = df_timeline_final.copy()

# -------------------------
# 1) Pago banco esperado
# -------------------------
df_timeline["Pago_banco_esperado"] = (
    df_timeline["D_BRAVO"]
    - (df_timeline["D_BRAVO"] * df_timeline["Descuento"])
)

# -------------------------
# 2) Ingreso esperado (CE = 0.15)
# -------------------------
CE = 0.15

df_timeline["Ingreso_esperado"] = np.maximum(
    0,
    (df_timeline["D_BRAVO"] - df_timeline["Pago_banco_esperado"]) * 1.19 * CE
)

# -------------------------
# 3) Mover columna al lado de D_BRAVO
# -------------------------
cols = df_timeline.columns.tolist()
idx = cols.index("D_BRAVO")

# quitamos y reinsertamos
cols.remove("Ingreso_esperado")
cols.insert(idx + 1, "Ingreso_esperado")

df_timeline = df_timeline[cols]

df_timeline[[
    "D_BRAVO",
    "Descuento",
    "Pago_banco_esperado",
    "Ingreso_esperado"
]].head()

,D_BRAVO,Descuento,Pago_banco_esperado,Ingreso_esperado
0,45488000.0,0.469694,2.412254e+07,3.813734e+06
1,1398270.0,0.305964,9.704500e+05,7.636587e+04
2,1084582.0,0.355043,6.995083e+05,6.873565e+04
3,3423000.0,0.536113,1.587884e+06,3.275682e+05
4,2532000.0,0.497194,1.273105e+06,2.247128e+05


In [22]:
df_timeline

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,CE,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,3.813734e+06,90.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.412254e+07
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,7.636587e+04,91.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.704500e+05
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,6.873565e+04,92.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.995083e+05
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,3.275682e+05,90.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587884e+06
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,2.247128e+05,102.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.273105e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44464,3206721655,2121952,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164459,274220.0,8.049991e+03,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.291220e+05
44465,3206721655,2121953,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164761,370308.0,1.089072e+04,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.092956e+05
44466,3206721655,2121954,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.166528,434983.0,1.293001e+04,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.625460e+05
44467,3108495298,2122000,1052379703,Edwin Augusto Medina Gallo,Julian Andres Ossa Bohorquez,Nu,0.085922,6615000.0,1.014551e+05,3.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.046624e+06


In [23]:
import pandas as pd
import numpy as np
import re
import unicodedata

df_liq_aux = df_liq.copy()

def _norm_col(x: str) -> str:
    # normaliza: lower, quita tildes, colapsa espacios
    x = str(x).replace("\u00a0", " ")
    x = re.sub(r"\s+", " ", x).strip().lower()
    x = "".join(ch for ch in unicodedata.normalize("NFD", x) if unicodedata.category(ch) != "Mn")
    return x

# --- 1) Encontrar el nombre REAL de columnas (robusto)
cols_norm = {c: _norm_col(c) for c in df_liq_aux.columns}

# deuda (lo que tú quieres para D_BRAVO) = "Deuda Resuelve"
col_deuda = next((c for c, cn in cols_norm.items() if cn in ("d_bravo", "deuda resuelve", "deuda_resuelve")), None)

# banco
col_banco = next((c for c, cn in cols_norm.items() if cn in ("bancos_estandar", "banco estandar", "banco_estandar")), None)

# id berex
col_berex = next((c for c, cn in cols_norm.items() if cn in ("deuda berex", "deuda_berex", "id deuda berex", "id_deuda_berex")), None)

print("Columna deuda detectada:", col_deuda)
print("Columna banco detectada:", col_banco)
print("Columna berex detectada:", col_berex)

if col_deuda is None:
    raise KeyError("No encontré la columna de deuda (esperaba 'Deuda Resuelve' o 'D_BRAVO').")
if col_banco is None:
    raise KeyError("No encontré la columna 'BANCOS_ESTANDAR' (ni variantes).")
if col_berex is None:
    raise KeyError("No encontré la columna 'Deuda Berex' (ni variantes).")

# --- 2) Construir Id deuda desde Deuda Berex
df_liq_aux["Id deuda"] = pd.to_numeric(df_liq_aux[col_berex], errors="coerce").astype("Int64")

# --- 3) Parse D_BRAVO a float (robusto)
s = df_liq_aux[col_deuda].astype(str).str.strip()

# limpiar símbolos y separar miles/decimales típico de COP
s = s.str.replace(r"[^\d,\.]", "", regex=True)

# caso típico: "45.488.000" -> quitar puntos miles
# y "45,488,000" -> quitar comas miles
# y si viene con decimal, lo intentamos preservar
# estrategia: si tiene ambos, asumimos decimal el último separador
def _parse_money_str(x):
    if x in ("", "nan", "None"):
        return np.nan
    if "," in x and "." in x:
        # el separador que aparezca más a la derecha es decimal
        if x.rfind(",") > x.rfind("."):
            x = x.replace(".", "")
            x = x.replace(",", ".")
        else:
            x = x.replace(",", "")
    else:
        # si solo tiene comas: puede ser miles o decimal
        if "," in x:
            parts = x.split(",")
            # si último bloque tiene 1-2 dígitos, decimal
            if len(parts[-1]) in (1,2):
                x = x.replace(".", "")
                x = x.replace(",", ".")
            else:
                x = x.replace(",", "")
        # si solo tiene puntos: miles o decimal
        if "." in x:
            parts = x.split(".")
            if len(parts[-1]) not in (1,2):  # no parece decimal
                x = x.replace(".", "")
    try:
        return float(x)
    except:
        return np.nan

df_liq_aux["Deuda_Resuelve_num"] = s.map(_parse_money_str)

# --- 4) Mapas por Id deuda
map_banco_liq = (
    df_liq_aux.dropna(subset=["Id deuda", col_banco])
             .drop_duplicates(subset=["Id deuda"])
             .set_index("Id deuda")[col_banco]
)

map_deuda_liq = (
    df_liq_aux.dropna(subset=["Id deuda", "Deuda_Resuelve_num"])
             .drop_duplicates(subset=["Id deuda"])
             .set_index("Id deuda")["Deuda_Resuelve_num"]
)

# --- 5) Rellenar SOLO filas Liquidación y SOLO si está nulo
df = df_timeline.copy()
mask_liq = df["tipo_fila"].astype("string").eq("Liquidación")

df.loc[mask_liq, "BANCOS_ESTANDAR"] = (
    df.loc[mask_liq, "BANCOS_ESTANDAR"]
      .fillna(df.loc[mask_liq, "Id deuda"].map(map_banco_liq))
)

df.loc[mask_liq, "D_BRAVO"] = (
    df.loc[mask_liq, "D_BRAVO"]
      .fillna(df.loc[mask_liq, "Id deuda"].map(map_deuda_liq))
)

df["D_BRAVO"] = pd.to_numeric(df["D_BRAVO"], errors="coerce").astype(float)

df_timeline_final = df

# --- 6) Chequeo
m = df_timeline_final["tipo_fila"].astype("string").eq("Liquidación")
print("Liquidación sin BANCOS_ESTANDAR:", df_timeline_final.loc[m, "BANCOS_ESTANDAR"].isna().sum())
print("Liquidación sin D_BRAVO:", df_timeline_final.loc[m, "D_BRAVO"].isna().sum())

Columna deuda detectada: Deuda Resuelve 
Columna banco detectada: BANCOS_ESTANDAR
Columna berex detectada: Deuda Berex
Liquidación sin BANCOS_ESTANDAR: 0
Liquidación sin D_BRAVO: 0


In [24]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,CE,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,3.813734e+06,90.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.412254e+07
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,7.636587e+04,91.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.704500e+05
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,6.873565e+04,92.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.995083e+05
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,3.275682e+05,90.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587884e+06
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,2.247128e+05,102.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.273105e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44464,3206721655,2121952,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164459,274220.0,8.049991e+03,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.291220e+05
44465,3206721655,2121953,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164761,370308.0,1.089072e+04,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.092956e+05
44466,3206721655,2121954,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.166528,434983.0,1.293001e+04,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.625460e+05
44467,3108495298,2122000,1052379703,Edwin Augusto Medina Gallo,Julian Andres Ossa Bohorquez,Nu,0.085922,6615000.0,1.014551e+05,3.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.046624e+06


In [25]:
# =========================================================
# BASE FUNNEL COMPLETA (1 fila por Id deuda) desde df_timeline_final
# Compatible con Google Colab y GitHub / ejecución local
# =========================================================

import pandas as pd
import numpy as np

# -----------------------------
# 0) Copia segura
# -----------------------------
df = df_timeline_final.copy()

# -----------------------------
# 1) Tipos / limpieza base
# -----------------------------
df["Id deuda"] = pd.to_numeric(df.get("Id deuda"), errors="coerce").astype("Int64")
df["Referencia"] = pd.to_numeric(df.get("Referencia"), errors="coerce").astype("Int64")

df["inserted_at"] = pd.to_datetime(df.get("inserted_at"), errors="coerce")

df["D_BRAVO"] = pd.to_numeric(df.get("D_BRAVO"), errors="coerce")
df["Ingreso_esperado"] = pd.to_numeric(df.get("Ingreso_esperado"), errors="coerce")
df["payment_to_bank"] = pd.to_numeric(df.get("payment_to_bank"), errors="coerce")

if "CATEGORIA_PRED" not in df.columns:
    raise ValueError("df_timeline_final no tiene la columna 'CATEGORIA_PRED'.")

df["_cat_all_norm"] = (
    df["CATEGORIA_PRED"]
    .astype("string")
    .str.strip()
    .str.upper()
)

# -----------------------------
# 2) Histórico liquidado
# -----------------------------
liq_hist = (
    df.groupby("Id deuda")["_cat_all_norm"]
      .apply(lambda s: (s == "LIQUIDADO").any())
)

# -----------------------------
# 3) Último registro por deuda
# -----------------------------
df = df.sort_values(["Id deuda", "inserted_at"], na_position="last")
df_ult = df.groupby("Id deuda", as_index=False).tail(1).copy()

df_ult = df_ult.rename(columns={
    "inserted_at": "inserted_at_ultima",
    "observations": "observations_ultima",
    "CATEGORIA_PRED": "CATEGORIA_PRED_ultima",
    "payment_to_bank": "payment_to_bank_ultima",
    "end": "end_ultima",
})

# -----------------------------
# 4) Flags observación / mes actual (Bogotá)
# -----------------------------
obs = (
    df_ult.get("observations_ultima", pd.Series(pd.NA, index=df_ult.index))
    .astype("string")
    .str.strip()
)
df_ult["tiene_obs"] = obs.notna() & (obs != "")

hoy = pd.Timestamp.now(tz="America/Bogota")
inicio_mes = hoy.replace(day=1, hour=0, minute=0, second=0, microsecond=0)

col = "inserted_at_ultima"
df_ult[col] = pd.to_datetime(df_ult[col], errors="coerce")

if df_ult[col].dt.tz is None:
    df_ult[col] = df_ult[col].dt.tz_localize(
        "America/Bogota",
        nonexistent="shift_forward",
        ambiguous="NaT"
    )
else:
    df_ult[col] = df_ult[col].dt.tz_convert("America/Bogota")

df_ult["es_este_mes"] = df_ult[col].notna() & (df_ult[col] >= inicio_mes)

# -----------------------------
# 5) Normalizar categoría última
# -----------------------------
df_ult["cat_norm"] = (
    df_ult["CATEGORIA_PRED_ultima"]
    .astype("string")
    .str.strip()
    .str.upper()
)

# -----------------------------
# 6) Liquidado definitivo
# -----------------------------
df_ult["tiene_liquidado_historico"] = df_ult["Id deuda"].map(liq_hist).fillna(False)
cond_f4 = df_ult["tiene_liquidado_historico"]

# -----------------------------
# 7) Fase 3 — Avance
# -----------------------------
es_avance_cat = df_ult["cat_norm"].isin(
    ["ACUERDO", "DESCUENTO", "CONTRAPROPUESTA"]
).fillna(False)

deuda = pd.to_numeric(df_ult["D_BRAVO"], errors="coerce")
pab = pd.to_numeric(df_ult["payment_to_bank_ultima"], errors="coerce")
cond_f3 = (es_avance_cat & ((deuda - pab) >= 10000)).fillna(False)

# -----------------------------
# 8) Fase 1 — Sin actualizar
# -----------------------------
cond_f1 = ((~df_ult["tiene_obs"]) | (~df_ult["es_este_mes"])).fillna(False)

# -----------------------------
# 9) Fase 2 — Actualizado
# -----------------------------
cond_f2 = ((~cond_f1) & (~cond_f3) & (~cond_f4)).fillna(False)

# -----------------------------
# 10) Asignar FASE
# -----------------------------
df_ult["FASE"] = np.select(
    [
        cond_f4.to_numpy(bool),
        cond_f3.to_numpy(bool),
        cond_f2.to_numpy(bool),
        cond_f1.to_numpy(bool),
    ],
    [
        "Fase 4 — Liquidado",
        "Fase 3 — Avance",
        "Fase 2 — Actualizado",
        "Fase 1 — Sin actualizar / antes de mes",
    ],
    default="Fase 2 — Actualizado"
)

# -----------------------------
# 11) STATUS
# -----------------------------
status_cat = df_ult["cat_norm"].str.title()

df_ult["STATUS"] = np.select(
    [
        cond_f4.to_numpy(bool),
        cond_f3.to_numpy(bool),
        cond_f2.to_numpy(bool),
        (~df_ult["tiene_obs"]).to_numpy(bool),
        (df_ult["tiene_obs"] & ~df_ult["es_este_mes"]).to_numpy(bool),
    ],
    [
        "Liquidado",
        status_cat,
        status_cat,
        "Sin actualización",
        "Actualizado antes",
    ],
    default=status_cat
)

# -----------------------------
# 12) Ingreso funnel
# -----------------------------
df_ult["Ingreso_funnel"] = np.where(
    cond_f4.to_numpy(bool),
    0,
    df_ult["Ingreso_esperado"].fillna(0)
)
# -----------------------------
# 13) Base final (con columnas requeridas)
# -----------------------------
cols_requeridas = [
    "Referencia",
    "Id deuda",
    "Cedula",
    "Nombre del cliente",
    "Negociador",
    "BANCOS_ESTANDAR",
    "Descuento",
    "D_BRAVO",
    "MORA"
    "Estructurable",
    "Potencial",
    "Meses en el Programa",
    "Tipo de Liquidacion",
    "Bucket",
    "Ahorro total",
    "Ahorro medio",
    "Por cobrar",
    "Potencial Credito",
    "Estado Deuda",
    "sub_estado_deuda",
    "estado_reparadora",
    "sub_estado_reparadora",
    "Mora_estructurado",
    "MORA_CREDITO",
    "Priority_level",
    "Ingreso_esperado",
    "Ultimo contacto",
    "fecha mensaje",
    "CE"
]

cols_eventos_ultima = [
    "inserted_at_ultima",
    "end_ultima",
    "CATEGORIA_PRED_ultima",
    "payment_to_bank_ultima",
    "observations_ultima",
]

cols_funnel = [
    "Ingreso_funnel",
    "FASE",
    "STATUS",
    "tiene_obs",
    "es_este_mes",
    "tiene_liquidado_historico",
]

# 1) Garantiza que existan (si no, NA)
for c in (cols_requeridas + cols_eventos_ultima + cols_funnel):
    if c not in df_ult.columns:
        df_ult[c] = pd.NA

# 2) Orden: primero requeridas, luego eventos, luego funnel, luego cualquier otra que exista
extras = [c for c in df_ult.columns if c not in (cols_requeridas + cols_eventos_ultima + cols_funnel)]
cols_salida = cols_requeridas + cols_eventos_ultima + cols_funnel + extras

df_base_funnel = df_ult[cols_salida].copy()

print("✅ df_base_funnel listo")
print("shape:", df_base_funnel.shape)
print("\nDistribución por FASE:")
print(df_base_funnel["FASE"].value_counts(dropna=False))

print("\nLiquidado (Fase 4) - filas:",
      (df_base_funnel["FASE"] == "Fase 4 — Liquidado").sum())

print(df_base_funnel.head(20))

✅ df_base_funnel listo
shape: (41490, 48)

Distribución por FASE:
FASE
Fase 1 — Sin actualizar / antes de mes    38234
Fase 2 — Actualizado                       2411
Fase 3 — Avance                             778
Fase 4 — Liquidado                           67
Name: count, dtype: int64

Liquidado (Fase 4) - filas: 67
    Referencia  Id deuda Cedula                     Nombre del cliente  \
0   3208859986      4720      -       LUIS ALBERTO RODRIGUEZ MANOSALVA   
1   3208859986      4722      -       LUIS ALBERTO RODRIGUEZ MANOSALVA   
2   3208859986      4725      -       LUIS ALBERTO RODRIGUEZ MANOSALVA   
3   3228472133      6539      -     CRISTHIAN GONZALO SINISTERRA PEREZ   
4   3228472133      6540      -     CRISTHIAN GONZALO SINISTERRA PEREZ   
5   3504614613      6828      -           ANTONIO MARCOS TAMAYO TORRES   
6   3504614613      6829      -           ANTONIO MARCOS TAMAYO TORRES   
7   3153844365      6959      -                   EDINSON JIMENEZ RIZO   
8   315384436

In [26]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,CE,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,3.813734e+06,90.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.412254e+07
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,7.636587e+04,91.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.704500e+05
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,6.873565e+04,92.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.995083e+05
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,3.275682e+05,90.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587884e+06
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,2.247128e+05,102.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.273105e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44464,3206721655,2121952,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164459,274220.0,8.049991e+03,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.291220e+05
44465,3206721655,2121953,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164761,370308.0,1.089072e+04,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.092956e+05
44466,3206721655,2121954,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.166528,434983.0,1.293001e+04,8.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.625460e+05
44467,3108495298,2122000,1052379703,Edwin Augusto Medina Gallo,Julian Andres Ossa Bohorquez,Nu,0.085922,6615000.0,1.014551e+05,3.0,...,0.15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.046624e+06


In [27]:
df_base_funnel.info()

<class 'pandas.DataFrame'>
Index: 41490 entries, 0 to 44468
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype                         
---  ------                     --------------  -----                         
 0   Referencia                 41490 non-null  Int64                         
 1   Id deuda                   41490 non-null  Int64                         
 2   Cedula                     41441 non-null  object                        
 3   Nombre del cliente         41445 non-null  object                        
 4   Negociador                 41488 non-null  str                           
 5   BANCOS_ESTANDAR            41490 non-null  object                        
 6   Descuento                  41447 non-null  float64                       
 7   D_BRAVO                    41490 non-null  float64                       
 8   MORAEstructurable          0 non-null      object                        
 9   Potencial                  41446 

In [28]:
df_timeline_final.info()

<class 'pandas.DataFrame'>
RangeIndex: 44469 entries, 0 to 44468
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype                         
---  ------                  --------------  -----                         
 0   Referencia              44469 non-null  Int64                         
 1   Id deuda                44469 non-null  Int64                         
 2   Cedula                  44398 non-null  object                        
 3   Nombre del cliente      44402 non-null  object                        
 4   Negociador              44467 non-null  str                           
 5   BANCOS_ESTANDAR         44469 non-null  object                        
 6   Descuento               44426 non-null  float64                       
 7   D_BRAVO                 44469 non-null  float64                       
 8   Ingreso_esperado        44426 non-null  float64                       
 9   MORA                    44402 non-null  float64              

In [29]:
# =====================================================
# Google Sheets en Colab + GitHub usando MI_JSON
# - En Colab: lee MI_JSON desde Secrets (userdata)
# - En GitHub / local: lee MI_JSON desde variable de entorno o archivo .json
# =====================================================

!pip install -q gspread gspread_dataframe

import os, json
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe

def _load_service_account_info():
    """
    Devuelve el dict del service account.
    Prioridad:
      1) Colab: userdata.get("MI_JSON")  (tu regla)
      2) Env var: MI_JSON (json string) o MI_JSON_PATH (ruta a .json)
      3) Archivo: service_account.json en el repo (si existe)
    """
    # 1) Colab (seguro si existe)
    try:
        from google.colab import userdata  # solo existe en Colab
        mi_json = userdata.get("MI_JSON")  # <-- tal como lo pediste
        if mi_json:
            return json.loads(mi_json) if isinstance(mi_json, str) else mi_json
    except Exception:
        pass

    # 2) GitHub/Local: env var MI_JSON (contenido json)
    mi_json_env = os.getenv("MI_JSON")
    if mi_json_env:
        return json.loads(mi_json_env)

    # 2b) GitHub/Local: env var MI_JSON_PATH (ruta a archivo json)
    mi_json_path = os.getenv("MI_JSON_PATH")
    if mi_json_path and os.path.exists(mi_json_path):
        with open(mi_json_path, "r", encoding="utf-8") as f:
            return json.load(f)

    # 3) Fallback: archivo en el repo (NO recomendado si es privado, pero útil local)
    default_path = "service_account.json"
    if os.path.exists(default_path):
        with open(default_path, "r", encoding="utf-8") as f:
            return json.load(f)

    raise RuntimeError(
        "No encontré credenciales. En Colab asegúrate de tener el Secret MI_JSON. "
        "En GitHub configura env var MI_JSON (json) o MI_JSON_PATH (ruta)."
    )

def get_gspread_client():
    sa_info = _load_service_account_info()
    return gspread.service_account_from_dict(sa_info)

# =====================================================
# Leer la hoja "Asignacion"
# =====================================================
SPREADSHEET_ID = "184zZcHRajscoJRyjMkkQjPh0qOlY3iVkQICKHl-iOyU"
NOMBRE_HOJA = "Asignacion"

gc = get_gspread_client()
sh = gc.open_by_key(SPREADSHEET_ID)
ws = sh.worksheet(NOMBRE_HOJA)

df_asignacion = get_as_dataframe(ws, evaluate_formulas=True).dropna(how="all")
df_asignacion.columns = df_asignacion.columns.str.strip()

df_asignacion

,Negociador,Tipo
0,Julian Andres Ossa Bohorquez,Back
1,Gabriela Saavedra Latorre,Back
2,Yithza Camila Paez Lopez,Estructurados
3,Angie Lizeth Cubides Neira,Premium
4,William Santiago Abril Esguerra,Premium
5,Vivian Caterin Rodriguez Verano,Premium
6,Rosa Yessenia Jimenez Lara,Premium
7,Niyiret Julio Santos,Premium
8,Maria Alejandra Bejarano Tumay,Premium
9,Maria Daniela Sarta Sanchez,Premium


In [30]:
df_timeline_final.info()

<class 'pandas.DataFrame'>
RangeIndex: 44469 entries, 0 to 44468
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype                         
---  ------                  --------------  -----                         
 0   Referencia              44469 non-null  Int64                         
 1   Id deuda                44469 non-null  Int64                         
 2   Cedula                  44398 non-null  object                        
 3   Nombre del cliente      44402 non-null  object                        
 4   Negociador              44467 non-null  str                           
 5   BANCOS_ESTANDAR         44469 non-null  object                        
 6   Descuento               44426 non-null  float64                       
 7   D_BRAVO                 44469 non-null  float64                       
 8   Ingreso_esperado        44426 non-null  float64                       
 9   MORA                    44402 non-null  float64              

Aquí empezamos con los Bucket

In [31]:
import numpy as np
import pandas as pd

# =========================================================
# 1) Descuento_Actualizacion
#    Si payment_to_bank no es nulo: ((payment_to_bank/D_BRAVO)-1)*-1
# =========================================================
df_timeline_final["Descuento_Actualizacion"] = np.where(
    df_timeline_final["payment_to_bank"].notna(),
    ((df_timeline_final["payment_to_bank"] / df_timeline_final["D_BRAVO"]) - 1) * -1,
    np.nan
)

# =========================================================
# 2) Tipo de Actividad (reglas)
# =========================================================
cat = df_timeline_final["CATEGORIA_PRED"]
desc = df_timeline_final["Descuento_Actualizacion"]

cats_efectiva = {"ACUERDO", "DESCUENTO", "EN_CREDITO", "Liquidado", "SALDOS", "DEPOSITO"}
rango_desc = desc.between(0.03, 0.98, inclusive="both")

# EFECTIVA
cond_efectiva = cat.isin(cats_efectiva) & rango_desc

# ACTUALIZADO_ALIANZAS
cond_act_alianzas = (cat == "ACTUALIZADO_ALIANZAS")

# EFECTIVA DESCUENTO ALIANZAS:
# payment_to_bank < D_BRAVO y (D_BRAVO - payment_to_bank) <= 5000
diff = df_timeline_final["D_BRAVO"] - df_timeline_final["payment_to_bank"]
cond_desc_alianzas = (
    cond_act_alianzas &
    (df_timeline_final["payment_to_bank"] < df_timeline_final["D_BRAVO"]) &
    (diff <= 5000)
)

df_timeline_final["Tipo de Actividad"] = np.select(
    [cond_efectiva, cond_desc_alianzas, cond_act_alianzas],
    ["EFECTIVA", "EFECTIVA DESCUENTO ALIANZAS", "EFECTIVA ALIANZAS"],
    default="NO EFECTIVA"
)

# =========================================================
# 3) Si payment_to_bank, CATEGORIA_PRED y observations son NaN -> Tipo de Actividad = NaN
# =========================================================
cond_nan_total = (
    df_timeline_final["payment_to_bank"].isna() &
    df_timeline_final["CATEGORIA_PRED"].isna() &
    df_timeline_final["observations"].isna()
)
df_timeline_final.loc[cond_nan_total, "Tipo de Actividad"] = np.nan

# =========================================================
# 4) Insertar ambas columnas al lado de 'observations'
#    (quedan inmediatamente después de observations, en este orden)
# =========================================================
idx_obs = df_timeline_final.columns.get_loc("observations")

# Pop + insert para asegurar ubicación/orden
col_da = df_timeline_final.pop("Descuento_Actualizacion")
col_ta = df_timeline_final.pop("Tipo de Actividad")

df_timeline_final.insert(idx_obs + 1, "Descuento_Actualizacion", col_da)
df_timeline_final.insert(idx_obs + 2, "Tipo de Actividad", col_ta)

In [32]:
df_base_funnel.columns

Index(['Referencia', 'Id deuda', 'Cedula', 'Nombre del cliente', 'Negociador',
       'BANCOS_ESTANDAR', 'Descuento', 'D_BRAVO', 'MORAEstructurable',
       'Potencial', 'Meses en el Programa', 'Tipo de Liquidacion', 'Bucket',
       'Ahorro total', 'Ahorro medio', 'Por cobrar', 'Potencial Credito',
       'Estado Deuda', 'sub_estado_deuda', 'estado_reparadora',
       'sub_estado_reparadora', 'Mora_estructurado', 'MORA_CREDITO',
       'Priority_level', 'Ingreso_esperado', 'Ultimo contacto',
       'fecha mensaje', 'CE', 'inserted_at_ultima', 'end_ultima',
       'CATEGORIA_PRED_ultima', 'payment_to_bank_ultima',
       'observations_ultima', 'Ingreso_funnel', 'FASE', 'STATUS', 'tiene_obs',
       'es_este_mes', 'tiene_liquidado_historico', 'MORA', 'Estructurable',
       'ultimo contacto', 'tipo_fila', 'Negociador liquidacion', 'Por?',
       'Pago_banco_esperado', '_cat_all_norm', 'cat_norm'],
      dtype='str')

In [33]:
import numpy as np
import pandas as pd

# =========================================================
# 1) Descuento_Actualizacion
#    Si payment_to_bank no es nulo: ((payment_to_bank/D_BRAVO)-1)*-1
# =========================================================
df_base_funnel["Descuento_Actualizacion"] = np.where(
    df_base_funnel["payment_to_bank_ultima"].notna(),
    ((df_base_funnel["payment_to_bank_ultima"] / df_base_funnel["D_BRAVO"]) - 1) * -1,
    np.nan
)

# =========================================================
# 2) Tipo de Actividad (reglas)
# =========================================================
cat = df_base_funnel["CATEGORIA_PRED_ultima"]
desc = df_base_funnel["Descuento_Actualizacion"]

cats_efectiva = {"ACUERDO", "DESCUENTO", "EN_CREDITO", "Liquidado", "SALDOS", "DEPOSITO"}
rango_desc = desc.between(0.03, 0.98, inclusive="both")

# EFECTIVA
cond_efectiva = cat.isin(cats_efectiva) & rango_desc

# ACTUALIZADO_ALIANZAS
cond_act_alianzas = (cat == "ACTUALIZADO_ALIANZAS")

# EFECTIVA DESCUENTO ALIANZAS:
# payment_to_bank < D_BRAVO y (D_BRAVO - payment_to_bank) <= 5000
diff = df_base_funnel["D_BRAVO"] - df_base_funnel["payment_to_bank_ultima"]
cond_desc_alianzas = (
    cond_act_alianzas &
    (df_base_funnel["payment_to_bank_ultima"] < df_base_funnel["D_BRAVO"]) &
    (diff <= 5000)
)

df_base_funnel["Tipo de Actividad"] = np.select(
    [cond_efectiva, cond_desc_alianzas, cond_act_alianzas],
    ["EFECTIVA", "EFECTIVA DESCUENTO ALIANZAS", "EFECTIVA ALIANZAS"],
    default="NO EFECTIVA"
)

# =========================================================
# 3) Si payment_to_bank, CATEGORIA_PRED y observations son NaN -> Tipo de Actividad = NaN
# =========================================================
cond_nan_total = (
    df_base_funnel["payment_to_bank_ultima"].isna() &
    df_base_funnel["CATEGORIA_PRED_ultima"].isna() &
    df_base_funnel["observations_ultima"].isna()
)
df_base_funnel.loc[cond_nan_total, "Tipo de Actividad"] = np.nan

# =========================================================
# 4) Insertar ambas columnas al lado de 'observations'
#    (quedan inmediatamente después de observations, en este orden)
# =========================================================
idx_obs = df_base_funnel.columns.get_loc("observations_ultima")

# Pop + insert para asegurar ubicación/orden
col_da = df_base_funnel.pop("Descuento_Actualizacion")
col_ta = df_base_funnel.pop("Tipo de Actividad")

df_base_funnel.insert(idx_obs + 1, "Descuento_Actualizacion", col_da)
df_base_funnel.insert(idx_obs + 2, "Tipo de Actividad", col_ta)

In [34]:
df_base_funnel = df_base_funnel.drop(columns=["Bucket"], errors="ignore")

In [35]:
df_timeline_final = df_timeline_final.drop(columns=["Bucket"], errors="ignore")

In [36]:
df_timeline_final.info()

<class 'pandas.DataFrame'>
RangeIndex: 44469 entries, 0 to 44468
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype                         
---  ------                   --------------  -----                         
 0   Referencia               44469 non-null  Int64                         
 1   Id deuda                 44469 non-null  Int64                         
 2   Cedula                   44398 non-null  object                        
 3   Nombre del cliente       44402 non-null  object                        
 4   Negociador               44467 non-null  str                           
 5   BANCOS_ESTANDAR          44469 non-null  object                        
 6   Descuento                44426 non-null  float64                       
 7   D_BRAVO                  44469 non-null  float64                       
 8   Ingreso_esperado         44426 non-null  float64                       
 9   MORA                     44402 non-null  float64  

In [37]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,end,payment_to_bank,CATEGORIA_PRED,observations,Descuento_Actualizacion,Tipo de Actividad,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,3.813734e+06,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.412254e+07
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,7.636587e+04,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.704500e+05
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,6.873565e+04,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.995083e+05
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,3.275682e+05,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587884e+06
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,2.247128e+05,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.273105e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44464,3206721655,2121952,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164459,274220.0,8.049991e+03,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.291220e+05
44465,3206721655,2121953,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164761,370308.0,1.089072e+04,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.092956e+05
44466,3206721655,2121954,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.166528,434983.0,1.293001e+04,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.625460e+05
44467,3108495298,2122000,1052379703,Edwin Augusto Medina Gallo,Julian Andres Ossa Bohorquez,Nu,0.085922,6615000.0,1.014551e+05,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.046624e+06


In [38]:
import numpy as np
import pandas as pd

# =========================================================
# CONFIG
# =========================================================
COL_ULT_CONTACTO = "ultimo contacto"   # <- así, con espacio
COL_NEGOCIADOR = "Negociador"          # <- AJUSTA si tu columna se llama distinto
COL_BANCO = "BANCOS_ESTANDAR"

# Distribución objetivo por negociador
TARGET_PCTS = {0: 0.25, 1: 0.20, 2: 0.25, 3: 0.10, 4: 0.10, 5: 0.10}
BUCKETS = [0, 1, 2, 3, 4, 5]

# Pesos para score final (puedes ajustarlos)
W_BASE = 0.75   # peso de lo ya construido (Bucket_row consolidado)
W_BANK = 0.25   # peso del banco
# Ahorro/D_BRAVO entra como impulso suave por buckets (ver más abajo)

# Más peso al mejor banco que al peor (para el criterio banco)
ALPHA_BEST_BANK = 0.70

# =========================================================
# 0) COPIA + LIMPIEZA / NORMALIZACIÓN
# =========================================================
df_timeline_final = df_timeline_final.copy()

# Numéricas
df_timeline_final[COL_ULT_CONTACTO] = pd.to_numeric(df_timeline_final[COL_ULT_CONTACTO], errors="coerce").fillna(6)
df_timeline_final["Potencial"] = pd.to_numeric(df_timeline_final["Potencial"], errors="coerce")
df_timeline_final["Estructurable"] = pd.to_numeric(df_timeline_final["Estructurable"], errors="coerce")
df_timeline_final["MORA"] = pd.to_numeric(df_timeline_final["MORA"], errors="coerce")
df_timeline_final["Meses en el Programa"] = pd.to_numeric(df_timeline_final["Meses en el Programa"], errors="coerce")
df_timeline_final["D_BRAVO"] = pd.to_numeric(df_timeline_final["D_BRAVO"], errors="coerce")
df_timeline_final["Ahorro total"] = pd.to_numeric(df_timeline_final["Ahorro total"], errors="coerce")

# Textos
mora_credito = df_timeline_final["MORA_CREDITO"].astype("string").str.strip().str.lower()
mora_estruct = df_timeline_final["Mora_estructurado"].astype("string").str.strip().str.lower()
cat_pred = df_timeline_final["CATEGORIA_PRED"].astype("string").str.strip()
cat_upper = cat_pred.str.upper()

# Potencial Credito mixto
pc_raw = df_timeline_final["Potencial Credito"].astype("string").str.strip()
pc_lower = pc_raw.str.lower()
pc_num = pd.to_numeric(pc_raw, errors="coerce")

# =========================================================
# 1) BUCKET "POR FILA" (PASO INTERMEDIO) => Bucket_row (0..5)
# =========================================================
# 1.1 Base por recencia (dinámico 0..5)
s = df_timeline_final[COL_ULT_CONTACTO]
base = pd.qcut(s.rank(method="first"), q=6, labels=False, duplicates="drop")
n_bins = int(base.max() + 1) if base.notna().any() else 1
if n_bins > 1:
    base_scaled = np.floor(base * (5 / (n_bins - 1))).astype(int)
else:
    base_scaled = pd.Series(0, index=df_timeline_final.index, dtype=int)
df_timeline_final["Bucket_row"] = base_scaled.astype("Int64")

# 1.2 Penalización por cliente con al menos 1 "Liquidado" (suave +1)
#clientes_con_liquidado = df_timeline_final.loc[cat_pred.eq("Liquidado"), "Referencia"].dropna().unique()
#mask_cliente_liquidado = df_timeline_final["Referencia"].isin(clientes_con_liquidado)
#df_timeline_final.loc[mask_cliente_liquidado, "Bucket_row"] = (
#    df_timeline_final.loc[mask_cliente_liquidado, "Bucket_row"].astype(int) + 1
#).clip(upper=5).astype("Int64")

# 1.3 Mora_estructurado
cond_estruct_si = mora_estruct.isin(["sí", "si"])
cond_estruct_no = mora_estruct.eq("no")

df_timeline_final.loc[cond_estruct_si, "Bucket_row"] = (
    df_timeline_final.loc[cond_estruct_si, "Bucket_row"].astype(int).clip(lower=4)
).astype("Int64")

df_timeline_final.loc[cond_estruct_no & ~cond_estruct_si, "Bucket_row"] = (
    df_timeline_final.loc[cond_estruct_no & ~cond_estruct_si, "Bucket_row"].astype(int).clip(lower=3)
).astype("Int64")

# 1.4 Potencial (alto aleja => peor (máximo))
p = df_timeline_final["Potencial"]
mask_p = p.notna()
if mask_p.any():
    p_bins = pd.qcut(p[mask_p].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_pbins = int(p_bins.max() + 1) if p_bins.notna().any() else 1
    if n_pbins > 1:
        p_scaled = np.floor(p_bins * (5 / (n_pbins - 1))).astype(int)
    else:
        p_scaled = pd.Series(0, index=p_bins.index, dtype=int)

    bucket_por_pot = pd.Series(pd.NA, index=df_timeline_final.index, dtype="Int64")
    bucket_por_pot.loc[p_scaled.index] = p_scaled.astype("Int64")

    df_timeline_final.loc[mask_p, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[mask_p, "Bucket_row"].astype(int),
        bucket_por_pot.loc[mask_p].astype(int),
    ).astype("Int64")

# 1.5 Estructurable (impulso suave)
mask_e = df_timeline_final["Estructurable"].isin([0, 1])
mask_e1 = mask_e & (df_timeline_final["Estructurable"] == 1)
mask_e0 = mask_e & (df_timeline_final["Estructurable"] == 0)

df_timeline_final.loc[mask_e1, "Bucket_row"] = (
    df_timeline_final.loc[mask_e1, "Bucket_row"].astype(int) - 1
).clip(lower=0).astype("Int64")

df_timeline_final.loc[mask_e0, "Bucket_row"] = (
    df_timeline_final.loc[mask_e0, "Bucket_row"].astype(int) + 1
).clip(upper=5).astype("Int64")

# 1.6 Potencial Credito (mixto)
mask_pc_num = pc_num.notna()
if mask_pc_num.any():
    pc_bins = pd.qcut(pc_num[mask_pc_num].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_pcbins = int(pc_bins.max() + 1) if pc_bins.notna().any() else 1
    if n_pcbins > 1:
        pc_scaled = np.floor(pc_bins * (5 / (n_pcbins - 1))).astype(int)
    else:
        pc_scaled = pd.Series(0, index=pc_bins.index, dtype=int)

    pc_score = pd.Series(pd.NA, index=df_timeline_final.index, dtype="Int64")
    pc_score.loc[pc_scaled.index] = pc_scaled.astype("Int64")

    # score 0-1 => mejora (-1)
    mask_pc_bajo = mask_pc_num & pc_score.isin([0, 1])
    df_timeline_final.loc[mask_pc_bajo, "Bucket_row"] = (
        df_timeline_final.loc[mask_pc_bajo, "Bucket_row"].astype(int) - 1
    ).clip(lower=0).astype("Int64")

    # score 4-5 => empeora (+1)
    mask_pc_alto = mask_pc_num & pc_score.isin([4, 5])
    df_timeline_final.loc[mask_pc_alto, "Bucket_row"] = (
        df_timeline_final.loc[mask_pc_alto, "Bucket_row"].astype(int) + 1
    ).clip(upper=5).astype("Int64")

# =========================================================
# ✅ CAMBIO ÚNICO AQUÍ:
# "Tradicional" => motiva Bucket 0 o 1 (máximo 1)
# =========================================================
mask_pc_trad = pc_lower.eq("tradicional")
df_timeline_final.loc[mask_pc_trad, "Bucket_row"] = (
    df_timeline_final.loc[mask_pc_trad, "Bucket_row"].astype(int).clip(upper=1)
).astype("Int64")

# (Dejamos NaN/vacíos sin empuje especial; quedan según otras reglas)

# 1.7 Meses en el Programa + MORA
mora = df_timeline_final["MORA"]
meses_prog = df_timeline_final["Meses en el Programa"]

# (A) MORA < 5 => grave => 4 o 5 dinámico
mask_mora_baja = mora.notna() & (mora < 5)
if mask_mora_baja.any():
    mora_baja = mora[mask_mora_baja]
    mb = pd.qcut(mora_baja.rank(method="first"), q=2, labels=False, duplicates="drop")
    if mb.notna().any() and int(mb.max() + 1) > 1:
        mb_scaled = mb.map({0: 4, 1: 5}).astype(int)
    else:
        mb_scaled = pd.Series(4, index=mora_baja.index, dtype=int)

    df_timeline_final.loc[mb_scaled.index, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[mb_scaled.index, "Bucket_row"].astype(int),
        mb_scaled.astype(int),
    ).astype("Int64")

# (B) MORA > 15 => intermedio => 3 o 4 dinámico
mask_mora_alta = mora.notna() & (mora > 15)
if mask_mora_alta.any():
    mora_alta = mora[mask_mora_alta]
    ma = pd.qcut(mora_alta.rank(method="first"), q=2, labels=False, duplicates="drop")
    if ma.notna().any() and int(ma.max() + 1) > 1:
        ma_scaled = ma.map({0: 3, 1: 4}).astype(int)
    else:
        ma_scaled = pd.Series(3, index=mora_alta.index, dtype=int)

    df_timeline_final.loc[ma_scaled.index, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[ma_scaled.index, "Bucket_row"].astype(int),
        ma_scaled.astype(int),
    ).astype("Int64")

# (C) MORA 6..15 y Meses>3 => impulso positivo -1
mask_top = mora.notna() & meses_prog.notna() & mora.between(6, 15, inclusive="both") & (meses_prog > 3)
df_timeline_final.loc[mask_top, "Bucket_row"] = (
    df_timeline_final.loc[mask_top, "Bucket_row"].astype(int) - 1
).clip(lower=0).astype("Int64")

# =========================================================
# 2) REGLAS DURAS (BUCKET 5 FIJO) — POR REFERENCIA
# =========================================================
cond_credito_si = mora_credito.isin(["sí", "si"])
refs_credito_si = df_timeline_final.loc[cond_credito_si, "Referencia"].dropna().unique()

refs_no_viable_demanda = df_timeline_final.loc[
    cat_upper.isin(["NO_VIABLE", "DEMANDA", "ILOCALIZADO", "PAGO_POR_FUERA"]),
    "Referencia"
].dropna().unique()

refs_hard_5 = set(refs_credito_si).union(set(refs_no_viable_demanda))

# =========================================================
# 3) BANCOS_ESTANDAR (POR REFERENCIA) — PEGA TU LISTA BUENO->MALO
# =========================================================
bancos_orden = [
    "Cobrando",
    "Rappipay",
    "Alkomprar",
    "Rappicard",
    "Pichincha",
    "Bancolombia",
    "Scotiabank Colpatria",
    "Rapicredit",
    "Bancoomeva",
    "Codensa",
    "Covinoc",
    "Zinobe",
    "Banco Popular",
    "SisteCredito",
    "Banco AV Villas",
    "Colsubsidio",
    "Refinancia",
    "Tuya",
    "Credijamar",
    "Agaval",
    "Alkosto",
    "Banco Falabella",
    "Banco de Bogotá",
    "Sistemcobro",
    "Bancamia",
    "Banco Finandina",
    "Itaú",
    "FGA",
    "BBVA Colombia",
    "Serlefin",
    "Flamingo",
    "Mundo Mujer",
    "Banco Davivienda",
    "Banco Caja Social",
    "Comultrasan",
    "Juriscoop",
    "Credivalores",
    "Banco de Occidente",
    "Compensar",
    "JOHN",
    "Fincomercio",
    "GNB Sudameris",
    "Serfinanza",
    "Nu",
    "LuloBank",
    "Aslegal",
    "Baninca",
    "Coltefinanciera",
    "Confiar",
    "Contacto solución",
    "Garantias Comunitarias",
    "GRUPO JURIDICO DEUDU",
    "Inversionistas Estratégicos",
    "Juancho te Presta",
    "QNT SAS",
]

bancos_norm = df_timeline_final[COL_BANCO].astype("string").str.strip().str.lower()

if len(bancos_orden) > 0:
    banco_rank_map = {b.strip().lower(): i for i, b in enumerate(bancos_orden)}
    rank_intermedio = len(bancos_orden) // 2
    df_timeline_final["_banco_rank"] = bancos_norm.map(banco_rank_map).fillna(rank_intermedio).astype(int)
else:
    df_timeline_final["_banco_rank"] = 0

grp_rank = df_timeline_final.groupby("Referencia")["_banco_rank"]
best_rank = grp_rank.min()
worst_rank = grp_rank.max()

score_banco = (ALPHA_BEST_BANK * best_rank) + ((1 - ALPHA_BEST_BANK) * worst_rank)

tmp = score_banco.rank(method="first")
bb = pd.qcut(tmp, q=6, labels=False, duplicates="drop")
n_bb = int(bb.max() + 1) if bb.notna().any() else 1
if n_bb > 1:
    bucket_banco = np.floor(bb * (5 / (n_bb - 1))).astype(int)
else:
    bucket_banco = pd.Series(0, index=score_banco.index, dtype=int)
bucket_banco = bucket_banco.clip(0, 5).astype("Int64")
bucket_banco.name = "Bucket_Banco"

# =========================================================
# 4) CRITERIO: max(Ahorro total / D_BRAVO) POR REFERENCIA
#    (alto ratio => empuja hacia bucket bajo, bajo ratio => hacia bucket alto)
# =========================================================
df_timeline_final["ratio_ahorro_db"] = np.where(
    (df_timeline_final["D_BRAVO"].notna()) & (df_timeline_final["D_BRAVO"] > 0) & df_timeline_final["Ahorro total"].notna(),
    df_timeline_final["Ahorro total"] / df_timeline_final["D_BRAVO"],
    np.nan
)

ratio_max_ref = df_timeline_final.groupby("Referencia")["ratio_ahorro_db"].max()

mask_ratio = ratio_max_ref.notna()
if mask_ratio.any():
    rr = pd.qcut(ratio_max_ref[mask_ratio].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_rr = int(rr.max() + 1) if rr.notna().any() else 1
    if n_rr > 1:
        rr_scaled = np.floor(rr * (5 / (n_rr - 1))).astype(int)
    else:
        rr_scaled = pd.Series(0, index=rr.index, dtype=int)

    bucket_ratio = pd.Series(pd.NA, index=ratio_max_ref.index, dtype="Int64")
    bucket_ratio.loc[rr_scaled.index] = (5 - rr_scaled).astype("Int64")  # invertido: alto ratio => bucket bajo
else:
    bucket_ratio = pd.Series(pd.NA, index=ratio_max_ref.index, dtype="Int64")

bucket_ratio.name = "Bucket_Ratio"

# =========================================================
# 5) TABLA POR REFERENCIA: SCORE CONTINUO (para ordenar dentro de negociador)
# =========================================================
bucket_ref_base = df_timeline_final.groupby("Referencia")["Bucket_row"].max().astype("Int64")
bucket_ref_base.name = "bucket_ref_base"

ref_table = pd.concat([bucket_ref_base, bucket_banco, bucket_ratio], axis=1)
ref_table.columns = ["bucket_ref_base", "Bucket_Banco", "Bucket_Ratio"]

# Fill intermedios si falta info (NO castiga ni premia)
ref_table["bucket_ref_base"] = ref_table["bucket_ref_base"].fillna(0)
ref_table["Bucket_Banco"] = ref_table["Bucket_Banco"].fillna(3)
ref_table["Bucket_Ratio"] = ref_table["Bucket_Ratio"].fillna(3)

# score continuo: menor = más importante
# (ratio ya viene como bucket 0..5 invertido, así que entra directo)
ref_table["score_ref"] = (
    (W_BASE * ref_table["bucket_ref_base"].astype(float)) +
    (W_BANK * ref_table["Bucket_Banco"].astype(float)) +
    (0.20 * ref_table["Bucket_Ratio"].astype(float))  # impulso ahorro/DBRAVO
)

# =========================================================
# 5.1) REGRA DURA: D_BRAVO MUY ALTO => BUCKET 0 o 1
# =========================================================

# promedio global de D_BRAVO
mean_dbravo = df_timeline_final["D_BRAVO"].mean()

# referencias con al menos una deuda 90% > promedio
refs_dbravo_alto = (
    df_timeline_final.loc[
        df_timeline_final["D_BRAVO"] >= 1.9 * mean_dbravo,
        "Referencia"
    ]
    .dropna()
    .unique()
)

# referencias con Banco Davivienda
refs_davivienda = (
    df_timeline_final.loc[
        df_timeline_final[COL_BANCO]
        .astype("string")
        .str.strip()
        .str.lower()
        .eq("banco davivienda"),
        "Referencia"
    ]
    .dropna()
    .unique()
)

# elegibles: alto D_BRAVO, no hard 5, no Davivienda
refs_dbravo_prioridad = (
    set(refs_dbravo_alto)
    - set(refs_hard_5)
    - set(refs_davivienda)
)

# aplicar regla:
# si bucket_base <= 2 => Bucket 0
# si bucket_base > 2  => Bucket 1
mask_prior = ref_table.index.isin(refs_dbravo_prioridad)

ref_table.loc[mask_prior & (ref_table["bucket_ref_base"] <= 2), "bucket_ref_base"] = 0
ref_table.loc[mask_prior & (ref_table["bucket_ref_base"] > 2), "bucket_ref_base"] = 1

# =========================================================
# 6) MAPEAR NEGOCIADOR POR REFERENCIA
#    (si una referencia aparece con varios negociadores, toma el más frecuente)
# =========================================================
ref_neg = (
    df_timeline_final.groupby(["Referencia", COL_NEGOCIADOR]).size()
    .reset_index(name="n")
    .sort_values(["Referencia", "n"], ascending=[True, False])
    .drop_duplicates("Referencia")
    .set_index("Referencia")[COL_NEGOCIADOR]
)

ref_table = ref_table.join(ref_neg.rename("negociador_ref"), how="left")

# =========================================================
# 7) ASIGNACIÓN EQUITATIVA POR NEGOCIADOR (CUOTAS)
#    Respeta hard=5. El resto se distribuye por score (mejor score => bucket más bajo).
# =========================================================
def _target_counts(n_total: int) -> dict:
    # floor + repartir sobrantes por los buckets más importantes primero
    raw = {b: int(np.floor(TARGET_PCTS[b] * n_total)) for b in BUCKETS}
    missing = n_total - sum(raw.values())
    # repartir sobrantes: 0,1,2,3,4,5
    for b in BUCKETS:
        if missing <= 0:
            break
        raw[b] += 1
        missing -= 1
    return raw

bucket_final = pd.Series(index=ref_table.index, dtype="Int64")

for neg, sub in ref_table.groupby("negociador_ref", dropna=False):
    refs = sub.index.tolist()
    n_total = len(refs)
    if n_total == 0:
        continue

    targets = _target_counts(n_total)

    # hard 5 dentro del negociador
    hard_mask = [r in refs_hard_5 for r in refs]
    hard_refs = sub.index[hard_mask]
    n_hard = len(hard_refs)

    # asignar hard primero
    if n_hard > 0:
        bucket_final.loc[hard_refs] = 5

    # refs elegibles (no hard)
    free = sub.loc[~pd.Index(refs).isin(hard_refs)].copy()
    n_free = len(free)

    if n_free == 0:
        continue

    # ajustar cuota bucket 5 para lo que falte (si hard ya ocupó parte)
    # si hard excede target5, dejamos bucket5 = hard y recortamos otros buckets desde el 4 hacia abajo
    if n_hard > targets[5]:
        extra = n_hard - targets[5]
        targets[5] = n_hard
        # recortar extra de 4,3,2,1,0 en ese orden
        for b in [4, 3, 2, 1, 0]:
            if extra <= 0:
                break
            take = min(targets[b], extra)
            targets[b] -= take
            extra -= take
    else:
        targets[5] = targets[5] - n_hard  # lo que queda por llenar con refs libres

    # si por redondeos quedó suma distinta a n_free, ajustar en bucket 4 (neutro)
    total_needed = sum(targets.values())
    if total_needed != n_free:
        diff = n_free - total_needed
        targets[4] = max(0, targets[4] + diff)

    # ordenar por score (menor=mejor)
    free = free.sort_values("score_ref", ascending=True)

    # asignar por bloques
    start = 0
    for b in BUCKETS:
        cnt = targets[b]
        if cnt <= 0:
            continue
        end = start + cnt
        bucket_final.loc[free.index[start:end]] = b
        start = end

# si alguna referencia quedó sin asignar (por negociador nulo o edge cases), cae a bucket por score global
mask_na = bucket_final.isna()
if mask_na.any():
    tmp = ref_table.loc[mask_na].sort_values("score_ref", ascending=True)
    # asignación simple por quantiles globales
    ranks = tmp["score_ref"].rank(method="first")
    q = pd.qcut(ranks, q=6, labels=False, duplicates="drop")
    n_q = int(q.max() + 1) if q.notna().any() else 1
    if n_q > 1:
        q_scaled = np.floor(q * (5 / (n_q - 1))).astype(int)
    else:
        q_scaled = pd.Series(0, index=tmp.index, dtype=int)
    bucket_final.loc[tmp.index] = q_scaled.clip(0, 5).astype("Int64")
    # hard sigue siendo 5
    bucket_final.loc[list(refs_hard_5)] = 5

ref_table["Bucket_ref"] = bucket_final.astype("Int64")

# =========================================================
# 8) PEGAR A TODAS LAS FILAS (UNA REFERENCIA = UN BUCKET)
# =========================================================
df_timeline_final = df_timeline_final.merge(
    ref_table[["Bucket_ref"]],
    left_on="Referencia",
    right_index=True,
    how="left",
)

df_timeline_final["Bucket"] = df_timeline_final["Bucket_ref"].astype("Int64")

# Limpieza auxiliares
df_timeline_final.drop(
    columns=["Bucket_ref", "Bucket_row", "_banco_rank", "ratio_ahorro_db"],
    inplace=True,
    errors="ignore",
)

df_timeline_final["Bucket"] = df_timeline_final["Bucket"].astype(int).clip(0, 5).astype("Int64")

##ESTE CODIGO PARA PLUS EL PROXIMO MES: import numpy as np
import pandas as pd

# =========================================================
# CONFIG
# =========================================================
COL_ULT_CONTACTO = "ultimo contacto"   # <- así, con espacio
COL_NEGOCIADOR = "Negociador"          # <- AJUSTA si tu columna se llama distinto
COL_BANCO = "BANCOS_ESTANDAR"

# Distribución objetivo por negociador (default)
TARGET_PCTS_DEFAULT = {0: 0.10, 1: 0.10, 2: 0.15, 3: 0.15, 4: 0.25, 5: 0.25}

# Distribución para Premium
TARGET_PCTS_PREMIUM = {0: 0.20, 1: 0.20, 2: 0.20, 3: 0.10, 4: 0.15, 5: 0.15}

BUCKETS = [0, 1, 2, 3, 4, 5]

# Pesos para score final (puedes ajustarlos)
W_BASE = 0.75   # peso de lo ya construido (Bucket_row consolidado)
W_BANK = 0.25   # peso del banco
# Ahorro/D_BRAVO entra como impulso suave por buckets (ver más abajo)

# Más peso al mejor banco que al peor (para el criterio banco)
ALPHA_BEST_BANK = 0.70


# =========================================================
# 0) MAPEO Negociador -> Tipo (desde df_asignacion)
#     (Premium vs resto)
# =========================================================
# df_asignacion debe tener columnas: "Negociador" y "Tipo"
neg_tipo_map = (
    df_asignacion[["Negociador", "Tipo"]]
    .dropna()
    .assign(
        Negociador=lambda x: x["Negociador"].astype(str).str.strip(),
        Tipo=lambda x: x["Tipo"].astype(str).str.strip(),
    )
    .drop_duplicates("Negociador")
    .set_index("Negociador")["Tipo"]
    .to_dict()
)


# =========================================================
# 0) COPIA + LIMPIEZA / NORMALIZACIÓN
# =========================================================
df_timeline_final = df_timeline_final.copy()

# Numéricas
df_timeline_final[COL_ULT_CONTACTO] = pd.to_numeric(df_timeline_final[COL_ULT_CONTACTO], errors="coerce").fillna(6)
df_timeline_final["Potencial"] = pd.to_numeric(df_timeline_final["Potencial"], errors="coerce")
df_timeline_final["Estructurable"] = pd.to_numeric(df_timeline_final["Estructurable"], errors="coerce")
df_timeline_final["MORA"] = pd.to_numeric(df_timeline_final["MORA"], errors="coerce")
df_timeline_final["Meses en el Programa"] = pd.to_numeric(df_timeline_final["Meses en el Programa"], errors="coerce")
df_timeline_final["D_BRAVO"] = pd.to_numeric(df_timeline_final["D_BRAVO"], errors="coerce")
df_timeline_final["Ahorro total"] = pd.to_numeric(df_timeline_final["Ahorro total"], errors="coerce")

# Textos
mora_credito = df_timeline_final["MORA_CREDITO"].astype("string").str.strip().str.lower()
mora_estruct = df_timeline_final["Mora_estructurado"].astype("string").str.strip().str.lower()
cat_pred = df_timeline_final["CATEGORIA_PRED"].astype("string").str.strip()
cat_upper = cat_pred.str.upper()

# Potencial Credito mixto
pc_raw = df_timeline_final["Potencial Credito"].astype("string").str.strip()
pc_lower = pc_raw.str.lower()
pc_num = pd.to_numeric(pc_raw, errors="coerce")

# =========================================================
# 1) BUCKET "POR FILA" (PASO INTERMEDIO) => Bucket_row (0..5)
# =========================================================
# 1.1 Base por recencia (dinámico 0..5)
s = df_timeline_final[COL_ULT_CONTACTO]
base = pd.qcut(s.rank(method="first"), q=6, labels=False, duplicates="drop")
n_bins = int(base.max() + 1) if base.notna().any() else 1
if n_bins > 1:
    base_scaled = np.floor(base * (5 / (n_bins - 1))).astype(int)
else:
    base_scaled = pd.Series(0, index=df_timeline_final.index, dtype=int)
df_timeline_final["Bucket_row"] = base_scaled.astype("Int64")

# 1.2 Penalización por cliente con al menos 1 "Liquidado" (suave +1)
clientes_con_liquidado = df_timeline_final.loc[cat_pred.eq("Liquidado"), "Referencia"].dropna().unique()
mask_cliente_liquidado = df_timeline_final["Referencia"].isin(clientes_con_liquidado)
df_timeline_final.loc[mask_cliente_liquidado, "Bucket_row"] = (
    df_timeline_final.loc[mask_cliente_liquidado, "Bucket_row"].astype(int) + 1
).clip(upper=5).astype("Int64")

# 1.3 Mora_estructurado
cond_estruct_si = mora_estruct.isin(["sí", "si"])
cond_estruct_no = mora_estruct.eq("no")

df_timeline_final.loc[cond_estruct_si, "Bucket_row"] = (
    df_timeline_final.loc[cond_estruct_si, "Bucket_row"].astype(int).clip(lower=4)
).astype("Int64")

df_timeline_final.loc[cond_estruct_no & ~cond_estruct_si, "Bucket_row"] = (
    df_timeline_final.loc[cond_estruct_no & ~cond_estruct_si, "Bucket_row"].astype(int).clip(lower=3)
).astype("Int64")

# 1.4 Potencial (alto aleja => peor (máximo))
p = df_timeline_final["Potencial"]
mask_p = p.notna()
if mask_p.any():
    p_bins = pd.qcut(p[mask_p].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_pbins = int(p_bins.max() + 1) if p_bins.notna().any() else 1
    if n_pbins > 1:
        p_scaled = np.floor(p_bins * (5 / (n_pbins - 1))).astype(int)
    else:
        p_scaled = pd.Series(0, index=p_bins.index, dtype=int)

    bucket_por_pot = pd.Series(pd.NA, index=df_timeline_final.index, dtype="Int64")
    bucket_por_pot.loc[p_scaled.index] = p_scaled.astype("Int64")

    df_timeline_final.loc[mask_p, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[mask_p, "Bucket_row"].astype(int),
        bucket_por_pot.loc[mask_p].astype(int),
    ).astype("Int64")

# 1.5 Estructurable (impulso suave)
mask_e = df_timeline_final["Estructurable"].isin([0, 1])
mask_e1 = mask_e & (df_timeline_final["Estructurable"] == 1)
mask_e0 = mask_e & (df_timeline_final["Estructurable"] == 0)

df_timeline_final.loc[mask_e1, "Bucket_row"] = (
    df_timeline_final.loc[mask_e1, "Bucket_row"].astype(int) - 1
).clip(lower=0).astype("Int64")

df_timeline_final.loc[mask_e0, "Bucket_row"] = (
    df_timeline_final.loc[mask_e0, "Bucket_row"].astype(int) + 1
).clip(upper=5).astype("Int64")

# 1.6 Potencial Credito (mixto)
mask_pc_num = pc_num.notna()
if mask_pc_num.any():
    pc_bins = pd.qcut(pc_num[mask_pc_num].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_pcbins = int(pc_bins.max() + 1) if pc_bins.notna().any() else 1
    if n_pcbins > 1:
        pc_scaled = np.floor(pc_bins * (5 / (n_pcbins - 1))).astype(int)
    else:
        pc_scaled = pd.Series(0, index=pc_bins.index, dtype=int)

    pc_score = pd.Series(pd.NA, index=df_timeline_final.index, dtype="Int64")
    pc_score.loc[pc_scaled.index] = pc_scaled.astype("Int64")

    # score 0-1 => mejora (-1)
    mask_pc_bajo = mask_pc_num & pc_score.isin([0, 1])
    df_timeline_final.loc[mask_pc_bajo, "Bucket_row"] = (
        df_timeline_final.loc[mask_pc_bajo, "Bucket_row"].astype(int) - 1
    ).clip(lower=0).astype("Int64")

    # score 4-5 => empeora (+1)
    mask_pc_alto = mask_pc_num & pc_score.isin([4, 5])
    df_timeline_final.loc[mask_pc_alto, "Bucket_row"] = (
        df_timeline_final.loc[mask_pc_alto, "Bucket_row"].astype(int) + 1
    ).clip(upper=5).astype("Int64")

# Tradicional o NaN => intermedio (mínimo 3)
mask_pc_trad = pc_lower.eq("tradicional")
mask_pc_nan = pc_raw.isna() | pc_lower.isin(["<na>", "nan", "none", ""])
mask_pc_intermedio = (mask_pc_trad | mask_pc_nan) & (~mask_pc_num)
df_timeline_final.loc[mask_pc_intermedio, "Bucket_row"] = (
    df_timeline_final.loc[mask_pc_intermedio, "Bucket_row"].astype(int).clip(lower=3)
).astype("Int64")

# 1.7 Meses en el Programa + MORA
mora = df_timeline_final["MORA"]
meses_prog = df_timeline_final["Meses en el Programa"]

# (A) MORA < 5 => grave => 4 o 5 dinámico
mask_mora_baja = mora.notna() & (mora < 5)
if mask_mora_baja.any():
    mora_baja = mora[mask_mora_baja]
    mb = pd.qcut(mora_baja.rank(method="first"), q=2, labels=False, duplicates="drop")
    if mb.notna().any() and int(mb.max() + 1) > 1:
        mb_scaled = mb.map({0: 4, 1: 5}).astype(int)
    else:
        mb_scaled = pd.Series(4, index=mora_baja.index, dtype=int)

    df_timeline_final.loc[mb_scaled.index, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[mb_scaled.index, "Bucket_row"].astype(int),
        mb_scaled.astype(int),
    ).astype("Int64")

# (B) MORA > 15 => intermedio => 3 o 4 dinámico
mask_mora_alta = mora.notna() & (mora > 15)
if mask_mora_alta.any():
    mora_alta = mora[mask_mora_alta]
    ma = pd.qcut(mora_alta.rank(method="first"), q=2, labels=False, duplicates="drop")
    if ma.notna().any() and int(ma.max() + 1) > 1:
        ma_scaled = ma.map({0: 3, 1: 4}).astype(int)
    else:
        ma_scaled = pd.Series(3, index=mora_alta.index, dtype=int)

    df_timeline_final.loc[ma_scaled.index, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[ma_scaled.index, "Bucket_row"].astype(int),
        ma_scaled.astype(int),
    ).astype("Int64")

# (C) MORA 6..15 y Meses>3 => impulso positivo -1
mask_top = mora.notna() & meses_prog.notna() & mora.between(6, 15, inclusive="both") & (meses_prog > 3)
df_timeline_final.loc[mask_top, "Bucket_row"] = (
    df_timeline_final.loc[mask_top, "Bucket_row"].astype(int) - 1
).clip(lower=0).astype("Int64")

# =========================================================
# 2) REGLAS DURAS (BUCKET 5 FIJO) — POR REFERENCIA
# =========================================================
cond_credito_si = mora_credito.isin(["sí", "si"])
refs_credito_si = df_timeline_final.loc[cond_credito_si, "Referencia"].dropna().unique()

refs_no_viable_demanda = df_timeline_final.loc[
    cat_upper.isin(["NO_VIABLE", "DEMANDA", "ILOCALIZADO", "PAGO_POR_FUERA"]),
    "Referencia"
].dropna().unique()

refs_hard_5 = set(refs_credito_si).union(set(refs_no_viable_demanda))

# =========================================================
# 3) BANCOS_ESTANDAR (POR REFERENCIA) — PEGA TU LISTA BUENO->MALO
# =========================================================
bancos_orden = [
    "Cobrando","Rappipay","Alkomprar","Rappicard","Pichincha","Bancolombia",
    "Scotiabank Colpatria","Rapicredit","Bancoomeva","Codensa","Covinoc","Zinobe",
    "Banco Popular","SisteCredito","Banco AV Villas","Colsubsidio","Refinancia","Tuya",
    "Credijamar","Agaval","Alkosto","Banco Falabella","Banco de Bogotá","Sistemcobro",
    "Bancamia","Banco Finandina","Itaú","FGA","BBVA Colombia","Serlefin","Flamingo",
    "Mundo Mujer","Banco Davivienda","Banco Caja Social","Comultrasan","Juriscoop",
    "Credivalores","Banco de Occidente","Compensar","JOHN","Fincomercio","GNB Sudameris",
    "Serfinanza","Nu","LuloBank","Aslegal","Baninca","Coltefinanciera","Confiar",
    "Contacto solución","Garantias Comunitarias","GRUPO JURIDICO DEUDU",
    "Inversionistas Estratégicos","Juancho te Presta","QNT SAS",
]

bancos_norm = df_timeline_final[COL_BANCO].astype("string").str.strip().str.lower()

if len(bancos_orden) > 0:
    banco_rank_map = {b.strip().lower(): i for i, b in enumerate(bancos_orden)}
    rank_intermedio = len(bancos_orden) // 2
    df_timeline_final["_banco_rank"] = bancos_norm.map(banco_rank_map).fillna(rank_intermedio).astype(int)
else:
    df_timeline_final["_banco_rank"] = 0

grp_rank = df_timeline_final.groupby("Referencia")["_banco_rank"]
best_rank = grp_rank.min()
worst_rank = grp_rank.max()

score_banco = (ALPHA_BEST_BANK * best_rank) + ((1 - ALPHA_BEST_BANK) * worst_rank)

tmp = score_banco.rank(method="first")
bb = pd.qcut(tmp, q=6, labels=False, duplicates="drop")
n_bb = int(bb.max() + 1) if bb.notna().any() else 1
if n_bb > 1:
    bucket_banco = np.floor(bb * (5 / (n_bb - 1))).astype(int)
else:
    bucket_banco = pd.Series(0, index=score_banco.index, dtype=int)
bucket_banco = bucket_banco.clip(0, 5).astype("Int64")
bucket_banco.name = "Bucket_Banco"

# =========================================================
# 4) CRITERIO: max(Ahorro total / D_BRAVO) POR REFERENCIA
# =========================================================
df_timeline_final["ratio_ahorro_db"] = np.where(
    (df_timeline_final["D_BRAVO"].notna()) & (df_timeline_final["D_BRAVO"] > 0) & df_timeline_final["Ahorro total"].notna(),
    df_timeline_final["Ahorro total"] / df_timeline_final["D_BRAVO"],
    np.nan
)

ratio_max_ref = df_timeline_final.groupby("Referencia")["ratio_ahorro_db"].max()

mask_ratio = ratio_max_ref.notna()
if mask_ratio.any():
    rr = pd.qcut(ratio_max_ref[mask_ratio].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_rr = int(rr.max() + 1) if rr.notna().any() else 1
    if n_rr > 1:
        rr_scaled = np.floor(rr * (5 / (n_rr - 1))).astype(int)
    else:
        rr_scaled = pd.Series(0, index=rr.index, dtype=int)

    bucket_ratio = pd.Series(pd.NA, index=ratio_max_ref.index, dtype="Int64")
    bucket_ratio.loc[rr_scaled.index] = (5 - rr_scaled).astype("Int64")
else:
    bucket_ratio = pd.Series(pd.NA, index=ratio_max_ref.index, dtype="Int64")

bucket_ratio.name = "Bucket_Ratio"

# =========================================================
# 5) TABLA POR REFERENCIA: SCORE CONTINUO
# =========================================================
bucket_ref_base = df_timeline_final.groupby("Referencia")["Bucket_row"].max().astype("Int64")
bucket_ref_base.name = "bucket_ref_base"

ref_table = pd.concat([bucket_ref_base, bucket_banco, bucket_ratio], axis=1)
ref_table.columns = ["bucket_ref_base", "Bucket_Banco", "Bucket_Ratio"]

ref_table["bucket_ref_base"] = ref_table["bucket_ref_base"].fillna(0)
ref_table["Bucket_Banco"] = ref_table["Bucket_Banco"].fillna(3)
ref_table["Bucket_Ratio"] = ref_table["Bucket_Ratio"].fillna(3)

ref_table["score_ref"] = (
    (W_BASE * ref_table["bucket_ref_base"].astype(float)) +
    (W_BANK * ref_table["Bucket_Banco"].astype(float)) +
    (0.20 * ref_table["Bucket_Ratio"].astype(float))
)

# =========================================================
# 5.1) REGRA DURA: D_BRAVO MUY ALTO => BUCKET 0 o 1
# =========================================================
mean_dbravo = df_timeline_final["D_BRAVO"].mean()

refs_dbravo_alto = (
    df_timeline_final.loc[df_timeline_final["D_BRAVO"] >= 1.9 * mean_dbravo, "Referencia"]
    .dropna().unique()
)

refs_davivienda = (
    df_timeline_final.loc[
        df_timeline_final[COL_BANCO].astype("string").str.strip().str.lower().eq("banco davivienda"),
        "Referencia"
    ]
    .dropna().unique()
)

refs_dbravo_prioridad = set(refs_dbravo_alto) - set(refs_hard_5) - set(refs_davivienda)

mask_prior = ref_table.index.isin(refs_dbravo_prioridad)
ref_table.loc[mask_prior & (ref_table["bucket_ref_base"] <= 2), "bucket_ref_base"] = 0
ref_table.loc[mask_prior & (ref_table["bucket_ref_base"] > 2), "bucket_ref_base"] = 1

# OJO: como bucket_ref_base cambió, re-calculamos score_ref
ref_table["score_ref"] = (
    (W_BASE * ref_table["bucket_ref_base"].astype(float)) +
    (W_BANK * ref_table["Bucket_Banco"].astype(float)) +
    (0.20 * ref_table["Bucket_Ratio"].astype(float))
)

# =========================================================
# 6) MAPEAR NEGOCIADOR POR REFERENCIA
# =========================================================
ref_neg = (
    df_timeline_final.groupby(["Referencia", COL_NEGOCIADOR]).size()
    .reset_index(name="n")
    .sort_values(["Referencia", "n"], ascending=[True, False])
    .drop_duplicates("Referencia")
    .set_index("Referencia")[COL_NEGOCIADOR]
)

ref_table = ref_table.join(ref_neg.rename("negociador_ref"), how="left")

# =========================================================
# 7) ASIGNACIÓN EQUITATIVA POR NEGOCIADOR (CUOTAS) + PREMIUM
# =========================================================
def _target_counts(n_total: int, pcts: dict) -> dict:
    raw = {b: int(np.floor(pcts[b] * n_total)) for b in BUCKETS}
    missing = n_total - sum(raw.values())
    for b in BUCKETS:
        if missing <= 0:
            break
        raw[b] += 1
        missing -= 1
    return raw

bucket_final = pd.Series(index=ref_table.index, dtype="Int64")

for neg, sub in ref_table.groupby("negociador_ref", dropna=False):
    refs = sub.index.tolist()
    n_total = len(refs)
    if n_total == 0:
        continue

    # Elegir distribución según tipo
    tipo_neg = neg_tipo_map.get(str(neg).strip(), "")
    pcts = TARGET_PCTS_PREMIUM if tipo_neg == "Premium" else TARGET_PCTS_DEFAULT

    targets = _target_counts(n_total, pcts)

    # hard 5 dentro del negociador
    hard_mask = [r in refs_hard_5 for r in refs]
    hard_refs = sub.index[hard_mask]
    n_hard = len(hard_refs)

    if n_hard > 0:
        bucket_final.loc[hard_refs] = 5

    free = sub.loc[~pd.Index(refs).isin(hard_refs)].copy()
    n_free = len(free)

    if n_free == 0:
        continue

    if n_hard > targets[5]:
        extra = n_hard - targets[5]
        targets[5] = n_hard
        for b in [4, 3, 2, 1, 0]:
            if extra <= 0:
                break
            take = min(targets[b], extra)
            targets[b] -= take
            extra -= take
    else:
        targets[5] = targets[5] - n_hard

    total_needed = sum(targets.values())
    if total_needed != n_free:
        diff = n_free - total_needed
        targets[4] = max(0, targets[4] + diff)

    free = free.sort_values("score_ref", ascending=True)

    start = 0
    for b in BUCKETS:
        cnt = targets[b]
        if cnt <= 0:
            continue
        end = start + cnt
        bucket_final.loc[free.index[start:end]] = b
        start = end

mask_na = bucket_final.isna()
if mask_na.any():
    tmp = ref_table.loc[mask_na].sort_values("score_ref", ascending=True)
    ranks = tmp["score_ref"].rank(method="first")
    q = pd.qcut(ranks, q=6, labels=False, duplicates="drop")
    n_q = int(q.max() + 1) if q.notna().any() else 1
    if n_q > 1:
        q_scaled = np.floor(q * (5 / (n_q - 1))).astype(int)
    else:
        q_scaled = pd.Series(0, index=tmp.index, dtype=int)
    bucket_final.loc[tmp.index] = q_scaled.clip(0, 5).astype("Int64")
    bucket_final.loc[list(refs_hard_5)] = 5

ref_table["Bucket_ref"] = bucket_final.astype("Int64")

# =========================================================
# 8) PEGAR A TODAS LAS FILAS (UNA REFERENCIA = UN BUCKET)
# =========================================================
df_timeline_final = df_timeline_final.merge(
    ref_table[["Bucket_ref"]],
    left_on="Referencia",
    right_index=True,
    how="left",
)

df_timeline_final["Bucket"] = df_timeline_final["Bucket_ref"].astype("Int64")

df_timeline_final.drop(
    columns=["Bucket_ref", "Bucket_row", "_banco_rank", "ratio_ahorro_db"],
    inplace=True,
    errors="ignore",
)

df_timeline_final["Bucket"] = df_timeline_final["Bucket"].astype(int).clip(0, 5).astype("Int64")


In [39]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,payment_to_bank,CATEGORIA_PRED,observations,Descuento_Actualizacion,Tipo de Actividad,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado,Bucket
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,3.813734e+06,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.412254e+07,1
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,7.636587e+04,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.704500e+05,1
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,6.873565e+04,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.995083e+05,1
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,3.275682e+05,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.587884e+06,2
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,2.247128e+05,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.273105e+06,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44464,3206721655,2121952,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164459,274220.0,8.049991e+03,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.291220e+05,0
44465,3206721655,2121953,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164761,370308.0,1.089072e+04,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.092956e+05,0
44466,3206721655,2121954,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.166528,434983.0,1.293001e+04,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.625460e+05,0
44467,3108495298,2122000,1052379703,Edwin Augusto Medina Gallo,Julian Andres Ossa Bohorquez,Nu,0.085922,6615000.0,1.014551e+05,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.046624e+06,5


In [40]:
import pandas as pd

# 1) Borrar columnas (no falla si alguna no existe)
df_base_funnel = df_base_funnel.drop(columns=["_cat_all_norm", "cat_norm"], errors="ignore")

# 2) Crear mapeo único Id deuda -> Bucket (sin duplicar filas)
#    Si hay repetidos, se queda con el primer Bucket no nulo (y si todos son nulos, queda nulo)
bucket_map = (
    df_timeline_final[["Id deuda", "Bucket"]]
    .dropna(subset=["Id deuda"])
    .sort_values("Id deuda")
    .drop_duplicates(subset=["Id deuda"], keep="first")
)

# 3) Agregar Bucket a df_base_funnel sin crear filas nuevas (LEFT JOIN)
df_base_funnel = df_base_funnel.merge(
    bucket_map,
    on="Id deuda",
    how="left",
    validate="m:1"  # muchas en df_base_funnel -> 1 en bucket_map
)

In [41]:
df_timeline_final.info()

<class 'pandas.DataFrame'>
RangeIndex: 44469 entries, 0 to 44468
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype                         
---  ------                   --------------  -----                         
 0   Referencia               44469 non-null  Int64                         
 1   Id deuda                 44469 non-null  Int64                         
 2   Cedula                   44398 non-null  object                        
 3   Nombre del cliente       44402 non-null  object                        
 4   Negociador               44467 non-null  str                           
 5   BANCOS_ESTANDAR          44469 non-null  object                        
 6   Descuento                44426 non-null  float64                       
 7   D_BRAVO                  44469 non-null  float64                       
 8   Ingreso_esperado         44426 non-null  float64                       
 9   MORA                     44402 non-null  float64  

In [42]:
df_base_funnel.info()

<class 'pandas.DataFrame'>
RangeIndex: 41490 entries, 0 to 41489
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype                         
---  ------                     --------------  -----                         
 0   Referencia                 41490 non-null  Int64                         
 1   Id deuda                   41490 non-null  Int64                         
 2   Cedula                     41441 non-null  object                        
 3   Nombre del cliente         41445 non-null  object                        
 4   Negociador                 41488 non-null  str                           
 5   BANCOS_ESTANDAR            41490 non-null  object                        
 6   Descuento                  41447 non-null  float64                       
 7   D_BRAVO                    41490 non-null  float64                       
 8   MORAEstructurable          0 non-null      object                        
 9   Potencial                  4

In [43]:
df_base_funnel

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORAEstructurable,Potencial,...,es_este_mes,tiene_liquidado_historico,MORA,Estructurable,ultimo contacto,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado,Bucket
0,3208859986,4720,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Pichincha,0.469694,45488000.0,<NA>,44.0,...,False,False,90.0,0.0,6.0,NaN,NaN,NaN,2.412254e+07,1
1,3208859986,4722,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Banco Falabella,0.305964,1398270.0,<NA>,3.0,...,False,False,91.0,1.0,6.0,NaN,NaN,NaN,9.704500e+05,1
2,3208859986,4725,-,LUIS ALBERTO RODRIGUEZ MANOSALVA,Laura Yineth Torres Moncada,Tuya,0.355043,1084582.0,<NA>,2.0,...,False,False,92.0,1.0,6.0,NaN,NaN,NaN,6.995083e+05,1
3,3228472133,6539,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,Tuya,0.536113,3423000.0,<NA>,30.0,...,False,False,90.0,0.0,6.0,NaN,NaN,NaN,1.587884e+06,2
4,3228472133,6540,-,CRISTHIAN GONZALO SINISTERRA PEREZ,Sergio Andres Polania Castellanos,BBVA Colombia,0.497194,2532000.0,<NA>,13.0,...,False,False,102.0,0.0,6.0,NaN,NaN,NaN,1.273105e+06,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41485,3206721655,2121952,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164459,274220.0,<NA>,1.0,...,False,False,8.0,1.0,0.0,NaN,NaN,NaN,2.291220e+05,0
41486,3206721655,2121953,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.164761,370308.0,<NA>,2.0,...,False,False,8.0,1.0,0.0,NaN,NaN,NaN,3.092956e+05,0
41487,3206721655,2121954,1113515560,LOURDES JOHANA SUAREZ RIOS,Lina Maria Peña Gordillo,SisteCredito,0.166528,434983.0,<NA>,2.0,...,False,False,8.0,1.0,0.0,NaN,NaN,NaN,3.625460e+05,0
41488,3108495298,2122000,1052379703,Edwin Augusto Medina Gallo,Julian Andres Ossa Bohorquez,Nu,0.085922,6615000.0,<NA>,30.0,...,False,False,3.0,0.0,6.0,NaN,NaN,NaN,6.046624e+06,5


In [44]:
import pandas as pd

# ====== AJUSTA ESTOS NOMBRES SI ES NECESARIO ======
COL_ID = "Id deuda"       # o "ID_DEUDA", "id_deuda", etc.
COL_REF = "Referencia"    # o "REFERENCIA", etc.

# Categorías
COL_BASE_CAT = "CATEGORIA_PRED_ultima"
COL_TIME_CAT = "CATEGORIA_PRED"

# ====== 1) Filtrar LIQUIDADO en ambos ======
base_liq = df_base_funnel.loc[df_base_funnel[COL_BASE_CAT].eq("Liquidado")].copy()
time_liq = df_timeline_final.loc[df_timeline_final[COL_TIME_CAT].eq("Liquidado")].copy()

# ====== 2) Ver LIQUIDADOS que están en timeline pero NO aparecen en base (por ID) ======
ids_base = set(base_liq[COL_ID].dropna().astype(str))
ids_time = set(time_liq[COL_ID].dropna().astype(str))

ids_liq_faltan_en_base = sorted(ids_time - ids_base)

df_ids_faltan = (
    time_liq.loc[time_liq[COL_ID].astype(str).isin(ids_liq_faltan_en_base),
                 [COL_ID, COL_REF, COL_TIME_CAT]]
    .drop_duplicates()
    .sort_values([COL_ID, COL_REF], na_position="last")
)

print("Conteo Liquidado base:", len(base_liq))
print("Conteo Liquidado timeline:", len(time_liq))
print("IDs Liquidado en timeline que NO están en base_liq:", len(ids_liq_faltan_en_base))

# ====== 3) Lo mismo por Referencia ======
refs_base = set(base_liq[COL_REF].dropna().astype(str))
refs_time = set(time_liq[COL_REF].dropna().astype(str))

refs_liq_faltan_en_base = sorted(refs_time - refs_base)

df_refs_faltan = (
    time_liq.loc[time_liq[COL_REF].astype(str).isin(refs_liq_faltan_en_base),
                 [COL_REF, COL_ID, COL_TIME_CAT]]
    .drop_duplicates()
    .sort_values([COL_REF, COL_ID], na_position="last")
)

print("Referencias Liquidado en timeline que NO están en base_liq:", len(refs_liq_faltan_en_base))

# ====== 4) (Opcional) Caso más estricto: existen en base, pero NO están como Liquidado en base ======
# IDs que están liquidado en timeline, pero en base aparecen con otra categoría (o múltiples)
base_map = (
    df_base_funnel[[COL_ID, COL_BASE_CAT]]
    .dropna(subset=[COL_ID])
    .assign(**{COL_ID: df_base_funnel[COL_ID].astype(str)})
)

# ids de timeline liquidado que sí existen en base pero base no los tiene liquidado (en ninguna fila)
ids_existentes_en_base = ids_time.intersection(set(base_map[COL_ID]))
ids_base_liq = set(base_liq[COL_ID].dropna().astype(str))

ids_liq_en_time_pero_no_liq_en_base = sorted(ids_existentes_en_base - ids_base_liq)

df_ids_no_liq_en_base = (
    df_timeline_final.loc[
        df_timeline_final[COL_TIME_CAT].eq("Liquidado") &
        df_timeline_final[COL_ID].dropna().astype(str).isin(ids_liq_en_time_pero_no_liq_en_base),
        [COL_ID, COL_REF, COL_TIME_CAT]
    ]
    .drop_duplicates()
)

print("IDs Liquidado en timeline que existen en base, pero base NO los tiene como Liquidado:",
      len(ids_liq_en_time_pero_no_liq_en_base))

# ====== RESULTADOS ======
# df_ids_faltan -> "Liquidado" en timeline pero no en base_liq (por ID)
# df_refs_faltan -> "Liquidado" en timeline pero no en base_liq (por Referencia)
# df_ids_no_liq_en_base -> "Liquidado" en timeline y existe en base, pero base no lo marca Liquidado

Conteo Liquidado base: 45
Conteo Liquidado timeline: 67
IDs Liquidado en timeline que NO están en base_liq: 22
Referencias Liquidado en timeline que NO están en base_liq: 17
IDs Liquidado en timeline que existen en base, pero base NO los tiene como Liquidado: 22


In [45]:
df_ids_faltan

,Id deuda,Referencia,CATEGORIA_PRED
10694,1689611,3173070507,Liquidado
18313,1843596,3125598964,Liquidado
19442,1857537,3172444857,Liquidado
19872,1862483,3122756298,Liquidado
20104,1866655,3219878250,Liquidado
21632,1884428,72150976,Liquidado
24166,1912675,3132762146,Liquidado
24642,1918196,3144592430,Liquidado
25004,1922348,3012040498,Liquidado
25517,1926433,3016061702,Liquidado


In [46]:
df_ids_no_liq_en_base

,Id deuda,Referencia,CATEGORIA_PRED
10694,1689611,3173070507,Liquidado
18313,1843596,3125598964,Liquidado
19442,1857537,3172444857,Liquidado
19872,1862483,3122756298,Liquidado
20104,1866655,3219878250,Liquidado
21632,1884428,72150976,Liquidado
24166,1912675,3132762146,Liquidado
24642,1918196,3144592430,Liquidado
25004,1922348,3012040498,Liquidado
25517,1926433,3016061702,Liquidado


#Subir dataframe

In [47]:
# =========================================================
# TIMELINE -> SUBIDA COMPLETA (robusta)  ✅
# - Con retry + backoff
# - Con chunks fallback
# - FIX: worksheet.update() con named args (compat GitHub/Colab)
# =========================================================

import os, json, re, time, random
import numpy as np
import pandas as pd

import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from gspread.exceptions import APIError, WorksheetNotFound

RETRIABLE_CODES = ["[500]", "[502]", "[503]", "[504]", "[429]"]

def _retry(fn, label="", tries=10, base_sleep=1.5, jitter=0.6, max_sleep=45):
    last_err = None
    for i in range(tries):
        try:
            return fn()
        except APIError as e:
            last_err = e
            msg = str(e)
            if any(c in msg for c in RETRIABLE_CODES):
                extra = 6.0 if ("resize" in label.lower() or "update" in label.lower()) else 0.0
                sleep_s = min((base_sleep + extra) * (2 ** i) + random.uniform(0, jitter), max_sleep)
                print(f"[RETRY {i+1}/{tries}] {label} -> {msg[:120]}... sleep {sleep_s:.1f}s")
                time.sleep(sleep_s)
                continue
            raise
    raise last_err

def _robust_json_loads(s: str) -> dict:
    if not isinstance(s, str):
        raise TypeError("Se esperaba string para parsear JSON")
    s0 = s.strip()
    if (s0.startswith('"') and s0.endswith('"')) or (s0.startswith("'") and s0.endswith("'")):
        s0 = s0[1:-1].strip()
    try:
        return json.loads(s0)
    except Exception:
        pass
    s1 = s0.replace("\\n", "\n")
    try:
        return json.loads(s1)
    except Exception:
        pass

    def fix_private_key(text: str) -> str:
        m = re.search(r'"private_key"\s*:\s*"([\s\S]*?)"\s*,\s*"client_email"', text)
        if not m:
            return text
        pk = m.group(1)
        pk_fixed = pk.replace("\n", "\\n")
        return text.replace(pk, pk_fixed)

    s2 = fix_private_key(s0)
    return json.loads(s2)

def load_service_account_info():
    try:
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json:
            if isinstance(mi_json, dict):
                return mi_json
            return _robust_json_loads(mi_json)
    except Exception:
        pass

    sa = os.getenv("GOOGLE_SERVICE_ACCOUNT_JSON") or os.getenv("MI_JSON")
    if not sa:
        raise ValueError("❌ Faltan credenciales: define GOOGLE_SERVICE_ACCOUNT_JSON (o MI_JSON).")
    return _robust_json_loads(sa)

def _prepare_df_for_sheets(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    df_out = df_out.replace([np.inf, -np.inf], np.nan)

    for c in df_out.columns:
        if pd.api.types.is_datetime64_any_dtype(df_out[c]):
            df_out[c] = df_out[c].dt.strftime("%Y-%m-%d %H:%M:%S")

    df_out = df_out.where(pd.notna(df_out), "")
    return df_out

def _upload_chunked(ws, df_out: pd.DataFrame, sheet_name: str, chunk_size: int = 3000, pause_s: float = 0.6):
    ncols = max(len(df_out.columns), 1)
    header = [df_out.columns.tolist()]

    _retry(lambda: ws.update(range_name="A1", values=header), label=f"update header {sheet_name}")

    data = df_out.values.tolist()
    start_row = 2
    last_col_letter = gspread.utils.rowcol_to_a1(1, ncols).replace("1", "")

    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        end_row = start_row + len(chunk) - 1
        cell_range = f"A{start_row}:{last_col_letter}{end_row}"

        _retry(
            lambda cr=cell_range, ch=chunk: ws.update(
                range_name=cr,
                values=ch,
                value_input_option="USER_ENTERED",
            ),
            label=f"update chunk {sheet_name} rows {start_row}-{end_row}"
        )

        start_row = end_row + 1
        time.sleep(pause_s)

def upload_timeline_full(gc, spreadsheet_id: str, df_timeline_final: pd.DataFrame, sheet_name="Timeline"):
    spreadsheet = _retry(lambda: gc.open_by_key(spreadsheet_id), label="open spreadsheet")
    df_out = _prepare_df_for_sheets(df_timeline_final)

    nrows = max(len(df_out) + 1, 2)
    ncols = max(len(df_out.columns), 1)

    def _get_or_create_ws():
        try:
            return spreadsheet.worksheet(sheet_name)
        except WorksheetNotFound:
            return spreadsheet.add_worksheet(title=sheet_name, rows="100", cols=str(ncols))

    ws = _retry(_get_or_create_ws, label=f"get/create worksheet {sheet_name}")

    # Timeline sí se puede reescribir completo
    _retry(lambda: ws.clear(), label=f"clear {sheet_name}")

    try:
        _retry(lambda: ws.resize(rows=1000, cols=ncols), label=f"resize small {sheet_name}", tries=10, base_sleep=2.0)
        _retry(lambda: ws.resize(rows=nrows, cols=ncols), label=f"resize final {sheet_name}", tries=10, base_sleep=2.0)
    except APIError as e:
        print(f"⚠️ Resize falló en '{sheet_name}'. Sigo sin resize. {str(e)[:140]}...")

    try:
        _retry(
            lambda: set_with_dataframe(ws, df_out, include_index=False, include_column_header=True, resize=False),
            label=f"set_with_dataframe {sheet_name}",
            tries=8,
            base_sleep=1.5
        )
        return True, f"✅ {sheet_name}: {len(df_out):,} filas x {ncols} cols (set_with_dataframe)"
    except APIError as e:
        print(f"⚠️ set_with_dataframe falló en '{sheet_name}'. Paso a chunks. {str(e)[:140]}...")

    _upload_chunked(ws, df_out, sheet_name, chunk_size=3000, pause_s=0.6)
    return True, f"✅ {sheet_name}: {len(df_out):,} filas x {ncols} cols (chunked)"


# --------- AUTH + RUN ----------
SERVICE_ACCOUNT_INFO = load_service_account_info()
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_info(SERVICE_ACCOUNT_INFO, scopes=SCOPES)
gc = gspread.authorize(creds)

SPREADSHEET_ID = "1Bm1wjsfXdNDFrFTStQJHkERC08Eo21BwjZnu-WncibY"

ok_t, msg_t = upload_timeline_full(gc, SPREADSHEET_ID, df_timeline_final, "Timeline")
print(msg_t)

✅ Timeline: 44,469 filas x 40 cols (set_with_dataframe)


In [48]:
# =========================================================
# FUNNEL -> UPSERT POR "Id deuda" ✅
# - NO borra la hoja
# - Lee lo existente en Sheets
# - Por cada Id deuda:
#     - si existe y alguna columna distinta -> actualiza la fila completa
#     - si no existe -> agrega al final
# - FIX: worksheet.update() con named args (compat GitHub/Colab)
# =========================================================

import os, json, re, time, random
import numpy as np
import pandas as pd

import gspread
from google.oauth2.service_account import Credentials
from gspread.exceptions import APIError, WorksheetNotFound

RETRIABLE_CODES = ["[500]", "[502]", "[503]", "[504]", "[429]"]

def _retry(fn, label="", tries=10, base_sleep=1.5, jitter=0.6, max_sleep=45):
    last_err = None
    for i in range(tries):
        try:
            return fn()
        except APIError as e:
            last_err = e
            msg = str(e)
            if any(c in msg for c in RETRIABLE_CODES):
                sleep_s = min((base_sleep) * (2 ** i) + random.uniform(0, jitter), max_sleep)
                print(f"[RETRY {i+1}/{tries}] {label} -> {msg[:120]}... sleep {sleep_s:.1f}s")
                time.sleep(sleep_s)
                continue
            raise
    raise last_err

def _robust_json_loads(s: str) -> dict:
    if not isinstance(s, str):
        raise TypeError("Se esperaba string para parsear JSON")
    s0 = s.strip()
    if (s0.startswith('"') and s0.endswith('"')) or (s0.startswith("'") and s0.endswith("'")):
        s0 = s0[1:-1].strip()
    try:
        return json.loads(s0)
    except Exception:
        pass
    s1 = s0.replace("\\n", "\n")
    try:
        return json.loads(s1)
    except Exception:
        pass

    def fix_private_key(text: str) -> str:
        m = re.search(r'"private_key"\s*:\s*"([\s\S]*?)"\s*,\s*"client_email"', text)
        if not m:
            return text
        pk = m.group(1)
        pk_fixed = pk.replace("\n", "\\n")
        return text.replace(pk, pk_fixed)

    s2 = fix_private_key(s0)
    return json.loads(s2)

def load_service_account_info():
    try:
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json:
            if isinstance(mi_json, dict):
                return mi_json
            return _robust_json_loads(mi_json)
    except Exception:
        pass

    sa = os.getenv("GOOGLE_SERVICE_ACCOUNT_JSON") or os.getenv("MI_JSON")
    if not sa:
        raise ValueError("❌ Faltan credenciales: define GOOGLE_SERVICE_ACCOUNT_JSON (o MI_JSON).")
    return _robust_json_loads(sa)

def _prepare_df_for_sheets(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    df_out = df_out.replace([np.inf, -np.inf], np.nan)

    for c in df_out.columns:
        if pd.api.types.is_datetime64_any_dtype(df_out[c]):
            # datetime64[ns] y datetime64[ns, tz]
            df_out[c] = pd.to_datetime(df_out[c], errors="coerce").dt.strftime("%Y-%m-%d %H:%M:%S")

    df_out = df_out.where(pd.notna(df_out), "")
    return df_out

def _df_to_str_matrix(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Normaliza para comparar contra Sheets (que casi siempre devuelve strings).
    """
    out = df[cols].copy()
    for c in cols:
        # todo como string comparable; "" para vacíos
        out[c] = out[c].astype(str)
        out.loc[out[c].isin(["nan", "NaT", "<NA>", "None"]), c] = ""
    return out

def _get_or_create_ws(spreadsheet, sheet_name: str, ncols: int):
    try:
        return spreadsheet.worksheet(sheet_name)
    except WorksheetNotFound:
        return spreadsheet.add_worksheet(title=sheet_name, rows="100", cols=str(ncols))

def _write_header_if_needed(ws, cols: list[str]):
    values = _retry(lambda: ws.get_all_values(), label="get_all_values header check")
    if not values:
        _retry(lambda: ws.update(range_name="A1", values=[cols]), label="write header (empty sheet)")
        return

    header = values[0]
    if header != cols:
        # IMPORTANTE: no reordenamos data vieja aquí, solo corregimos encabezado.
        # Si tu hoja tuviera columnas distintas, esto te lo hace evidente.
        print("⚠️ Header en Sheets distinto al DF. Se sobrescribe header con el del DF.")
        print("   Sheets:", header)
        print("   DF    :", cols)
        _retry(lambda: ws.update(range_name="A1", values=[cols]), label="rewrite header")

def _batch_update_rows(ws, start_col_letter: str, end_col_letter: str, row_blocks: list[tuple[int,int,list[list[str]]]]):
    """
    row_blocks: [(start_row, end_row, values_matrix)]
    """
    for (r1, r2, mat) in row_blocks:
        rng = f"{start_col_letter}{r1}:{end_col_letter}{r2}"
        _retry(
            lambda rr=rng, mm=mat: ws.update(range_name=rr, values=mm, value_input_option="USER_ENTERED"),
            label=f"update rows {r1}-{r2}"
        )
        time.sleep(0.4)

def _make_consecutive_blocks(rownums_sorted: list[int], values_by_rownum: dict[int, list[str]]):
    """
    Agrupa filas consecutivas para reducir llamadas a la API.
    Retorna [(start_row, end_row, matrix_values)]
    """
    blocks = []
    if not rownums_sorted:
        return blocks

    start = prev = rownums_sorted[0]
    mat = [values_by_rownum[start]]

    for r in rownums_sorted[1:]:
        if r == prev + 1:
            mat.append(values_by_rownum[r])
            prev = r
        else:
            blocks.append((start, prev, mat))
            start = prev = r
            mat = [values_by_rownum[r]]
    blocks.append((start, prev, mat))
    return blocks

def upload_funnel_upsert_by_id(gc, spreadsheet_id: str, df_base_funnel: pd.DataFrame, sheet_name="Funnel"):
    spreadsheet = _retry(lambda: gc.open_by_key(spreadsheet_id), label="open spreadsheet")
    df_out = _prepare_df_for_sheets(df_base_funnel)

    # Asegurar columnas EXACTAS y en el orden del DF
    cols = df_out.columns.tolist()
    if "Id deuda" not in cols:
        raise ValueError("❌ df_base_funnel no tiene la columna 'Id deuda'.")

    ncols = len(cols)
    ws = _retry(lambda: _get_or_create_ws(spreadsheet, sheet_name, ncols), label=f"get/create ws {sheet_name}")

    # Header
    _write_header_if_needed(ws, cols)

    # Leer todo lo existente
    values = _retry(lambda: ws.get_all_values(), label=f"get_all_values {sheet_name}")

    if len(values) <= 1:
        # Hoja vacía (o sólo header): subir todo sin borrar
        print("ℹ️ Funnel vacío (o sólo header). Hago carga inicial por append en chunks.")
        data_all = _df_to_str_matrix(df_out, cols).values.tolist()
        for i in range(0, len(data_all), 1000):
            chunk = data_all[i:i+1000]
            _retry(lambda ch=chunk: ws.append_rows(ch, value_input_option="USER_ENTERED"), label=f"append_rows init {i}-{i+len(chunk)-1}")
            time.sleep(0.7)
        return True, f"✅ {sheet_name}: carga inicial {len(df_out):,} filas x {ncols} cols (append)"

    sheet_header = values[0]
    sheet_rows = values[1:]

    # Construir DF de Sheets
    df_sheet = pd.DataFrame(sheet_rows, columns=sheet_header)

    # Si por cualquier razón Sheets no trae todas las columnas, nos quedamos con intersección
    missing_in_sheet = [c for c in cols if c not in df_sheet.columns]
    if missing_in_sheet:
        print("⚠️ En Sheets faltan estas columnas vs DF (ojo):", missing_in_sheet)
        # Igual seguimos, pero esas columnas las tratamos como "" al comparar
        for c in missing_in_sheet:
            df_sheet[c] = ""

    # Normalizar Id deuda como string (clave)
    df_sheet["Id deuda"] = df_sheet["Id deuda"].astype(str)
    df_py_cmp = _df_to_str_matrix(df_out, cols)
    df_py_cmp["Id deuda"] = df_py_cmp["Id deuda"].astype(str)

    # Map id -> row_number en Sheets (row 2 = primer dato)
    id_to_rownum = {}
    for i, v in enumerate(df_sheet["Id deuda"].tolist(), start=2):
        if v != "":
            # si hay duplicados, nos quedamos con el primero (puedes cambiarlo si prefieres el último)
            if v not in id_to_rownum:
                id_to_rownum[v] = i

    ids_sheet = set(id_to_rownum.keys())
    ids_py = set(df_py_cmp["Id deuda"].tolist())

    ids_common = ids_py.intersection(ids_sheet)
    ids_new = sorted(list(ids_py - ids_sheet))

    # Indexar para comparación rápida
    df_sheet_cmp = _df_to_str_matrix(df_sheet, cols)
    df_sheet_cmp["Id deuda"] = df_sheet_cmp["Id deuda"].astype(str)

    sheet_by_id = df_sheet_cmp.set_index("Id deuda", drop=False)
    py_by_id = df_py_cmp.set_index("Id deuda", drop=False)

    # Encontrar ids cambiados (comparando todas las columnas excepto Id deuda)
    compare_cols = [c for c in cols if c != "Id deuda"]
    changed_ids = []
    for _id in ids_common:
        # si por algo no aparece, saltamos
        if _id not in sheet_by_id.index or _id not in py_by_id.index:
            continue
        a = sheet_by_id.loc[_id, compare_cols]
        b = py_by_id.loc[_id, compare_cols]
        # a y b son Series
        if not a.equals(b):
            changed_ids.append(_id)

    # Preparar updates por row_number
    values_by_rownum = {}
    for _id in changed_ids:
        rownum = id_to_rownum.get(_id)
        if not rownum:
            continue
        row_values = py_by_id.loc[_id, cols].tolist()
        values_by_rownum[rownum] = row_values

    rownums_sorted = sorted(values_by_rownum.keys())

    start_col_letter = "A"
    end_col_letter = gspread.utils.rowcol_to_a1(1, ncols).replace("1", "")
    blocks = _make_consecutive_blocks(rownums_sorted, values_by_rownum)

    # Aplicar updates (en bloques)
    if blocks:
        _batch_update_rows(ws, start_col_letter, end_col_letter, blocks)

    # Append nuevos
    if ids_new:
        rows_new = [py_by_id.loc[_id, cols].tolist() for _id in ids_new]
        for i in range(0, len(rows_new), 1000):
            chunk = rows_new[i:i+1000]
            _retry(lambda ch=chunk: ws.append_rows(ch, value_input_option="USER_ENTERED"), label=f"append_rows new {i}-{i+len(chunk)-1}")
            time.sleep(0.8)

    return True, (
        f"✅ {sheet_name}: "
        f"{len(changed_ids):,} filas actualizadas, {len(ids_new):,} nuevas (Id deuda). "
        f"Total DF={len(df_out):,} / Total sheet aprox={len(sheet_rows):,}"
    )


# --------- AUTH + RUN ----------
SERVICE_ACCOUNT_INFO = load_service_account_info()
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_info(SERVICE_ACCOUNT_INFO, scopes=SCOPES)
gc = gspread.authorize(creds)

SPREADSHEET_ID = "1Bm1wjsfXdNDFrFTStQJHkERC08Eo21BwjZnu-WncibY"

ok_f, msg_f = upload_funnel_upsert_by_id(gc, SPREADSHEET_ID, df_base_funnel, "Funnel")
print(msg_f)

✅ Funnel: 41,490 filas actualizadas, 0 nuevas (Id deuda). Total DF=41,490 / Total sheet aprox=41,490
